In [ ]:
from tkinter import N
from typing import final
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd
import csv
import requests
import time
from datetime import datetime
import logging
from timeit import default_timer as timer
import numpy as np
import pandas as pd
from time import sleep

fail = []
# driver = webdriver.Chrome(executable_path='./chromedriver');
def scrape_a_bird(birdname,driver):
    try:
        driver.get('https://www.wikidata.org/wiki/Wikidata:Main_Page')
        input_avibase = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[placeholder='Search Wikidata']")))
        input_avibase.send_keys(birdname)
        input_avibase.send_keys(Keys.ENTER)
        sleep(1)
        xp = '//*[@id="redlist-js"]/div/div/div[2]/section/div[2]/article/a'
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xp)))
            driver.find_elements(by=By.XPATH, value=xp)[0].click()
        except Exception:
            sleep(1)
            new_xp = '//*[@id="redlist-js"]/div/div/div[2]/section/div[2]/article/a'
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, new_xp)))
                driver.find_elements(by=By.XPATH, value=new_xp)[0].click()
            except Exception:
                fail.append(birdname)
        finally:        
            # sleep(3)
            xp_expand = '//*[@id="details__toggle-all"]'
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xp_expand)))
                expand = driver.find_elements(by=By.XPATH, value=xp_expand)
                expand[0].click()
            except Exception:
                fail.append(birdname+':Expandfail')
            finally:
                sleep(1)
                try:
                    xp_year = '//*[@id="habitat-ecology"]/div[2]/div[1]/p[1]'
                    # xp_year = '//*[@id="habitat-ecology"]/div[2]/div[1]/p'
                    new_xp_year = '//*[@id="habitat-ecology"]/div[2]/div[1]/div/p'
                    xp_alt ='//*[@id="geographic-range"]/div[2]/div[2]/div/p[1]'
                    # xp_alt ='//*[@id="geographic-range"]/div[2]/div[2]/div/p'
                    new_xp_alt ='//*[@id="geographic-range"]/div[2]/div[2]/div/p'
                    xp_populate = '//*[@id="redlist-js"]/div/div[3]/div/div[1]/div[1]/p[2]'
                    # xp_populate = '//*[@id="redlist-js"]/div/div[3]/div/div[1]/div[1]/p'
                    new_xp_populate = '//*[@id="redlist-js"]/div/div[3]/div/div[1]/div[1]/p'
                    xp_habitat ='//*[@id="redlist-js"]/div/div[3]/div/div[1]/div[3]/p[2]'
                    # xp_habitat ='//*[@id="redlist-js"]/div/div[3]/div/div[1]/div[3]/p'
                    new_xp_habitat ='//*[@id="redlist-js"]/div/div[3]/div/div[1]/div[3]/p'
                    xp_sci_n_year = '//*[@id="taxonomy-details"]/div[1]/div[2]/p'
                    xp_conserv = '//*[@id="assessment-information"]/div[2]/div/p[1]/a/strong'
                    xp_sci_name = '//*[@id="taxonomy-details"]/div[1]/div[1]/p/span/em[1]'
                    # xp_sci_name = '//*[@id="taxonomy-details"]/div[1]/div[1]/p/span/em'
                    new_xp_sci_name = '//*[@id="taxonomy-details"]/div[1]/div[1]/p/span/em'
                    xp_order = '//*[@id="taxonomy"]/div[2]/div[1]/p/a/strong'
                    xp_family = '//*[@id="taxonomy"]/div[2]/div[2]/p/a/strong'
                    xp_genus = '//*[@id="taxonomy"]/div[2]/div[3]/p/a/strong'
                    xp_other_c_names = '//*[@id="taxonomy-details"]/div[2]/div[2]/div/p'
                    sci_name = driver.find_elements(by=By.XPATH, value=xp_sci_name)
                    d={'Name':[birdname],'IUCN_link':[driver.current_url]}
                    try:
                        d['sci_name'] = [sci_name[0].text]
                    except Exception:
                        new_xp_sci = '//*[@id="taxonomy-details"]/div[1]/div[2]/p'
                        try:
                            d['sci_name'] = [driver.find_elements(by=By.XPATH, value=new_xp_sci)[0].text]
                        except Exception:
                            d['sci_name'] = np.nan
                    order = driver.find_elements(by=By.XPATH, value=xp_order)
                    try:
                        d['order'] = [order[0].text]
                    except Exception:
                        d['order']= np.nan
                    family = driver.find_elements(by=By.XPATH, value=xp_family)
                    try:
                        d['family'] = [family[0].text]
                    except Exception:
                        d['family']= np.nan
                    genus = driver.find_elements(by=By.XPATH, value=xp_genus)
                    try:
                        d['genus'] = [genus[0].text]
                    except Exception:
                        d['genus']= np.nan
                    sci_n_year = driver.find_elements(by=By.XPATH, value=xp_sci_n_year)
                    try:
                        d['sci_n_year'] = [sci_n_year[0].text]
                    except Exception:
                        d['sci_n_year']= np.nan
                    year = driver.find_elements(by=By.XPATH, value=xp_year)
                    try:
                        d['year'] = [year[0].text]
                    except Exception:
                        d['year']= np.nan
                    altitude = driver.find_elements(by=By.XPATH, value=xp_alt)
                    try:
                        d['altitude'] = [altitude[0].text]
                    except Exception:
                        d['altitude']= np.nan
                    populate = driver.find_elements(by=By.XPATH, value=xp_populate)
                    try:
                        d['populate'] = [populate[0].text]
                    except Exception:
                        d['populate']= np.nan
                    habitat = driver.find_elements(by=By.XPATH, value=xp_habitat)
                    try:
                        d['habitat'] =[habitat[0].text]
                    except Exception:
                        d['habitat']= np.nan
                    conservation =  driver.find_elements(by=By.XPATH, value=xp_conserv)
                    try:
                        d['conservation'] = [conservation[0].text]
                    except Exception:
                        d['conservation']= np.nan
                    other_names = driver.find_elements(by=By.XPATH, value=xp_other_c_names)
                    try:
                        d['other_names'] = [other_names[0].text]
                    except Exception:
                        d['other_names'] = np.nan
                    # print(d)
                    idf = pd.DataFrame(d)
                    return idf
                except Exception:
                    fail.append(birdname)
                    return pd.DataFrame(d)
    except Exception:
        fail.append(birdname)
                
def iucn(fro,to,driver,data_df,list_of_birds):
    for bird_name in list_of_birds[fro:to]:
        try:
            data_df = pd.concat([data_df,scrape_a_bird(bird_name,driver)] )
            logging.debug('[+] '+ bird_name)
            print('[+] '+ bird_name)
        except Exception:
            logging.error('Unknown Exceptions for Bird-' + bird_name)
    return data_df

# # if __name__ == '__main__':
# startTime = datetime.now()
# birds = pd.DataFrame(columns=['Name','sci_name','order','family','genus','sci_n_year','year','altitude','populate','habitat','conservation','other_names','IUCN_link'])
# # driver = webdriver.Chrome('./chromedriver.exe')
# dz = pd.read_excel("assets/DataZone/HBW-BirdLife_List_of_Birds_v6.xlsx")
# list_of_birds = list(dz.loc[:,'Common name'])
# # f = int(input('from :'))
# # t = int(input('to :'))
# f = 8888
# t = 8891
# ds = 'iucn_'+str(f)+'_'+str(t)
# logging.basicConfig(filename="assets/rescrape_iucn/Log_"+ds+".log",format='%(asctime)s - %(levelname)s - %(message)s',filemode='a')
# birds = iucn(f,t,driver,birds,list_of_birds)
# birds.to_pickle("assets/rescrape_iucn/Pick_"+ds+".pkl")
# birds.to_csv("assets/rescrape_iucn/csv_"+ds+".csv")
# logging.info('\n\nTime taken : ',datetime.now() - startTime)
# print('falied birds : ',fail)
# print('\n\nTime taken : ',datetime.now() - startTime) 

# driver.quit()    
'''
conda activate indicwiki & cd Documents\birds & python Eol_Scraper_Optim.py
<f>
<t>
'''
if __name__ == '__main__':
    startTime = datetime.now()
    birds = pd.DataFrame(columns=['Name','sci_name','order','family','genus','sci_n_year','year','altitude','populate','habitat','conservation','other_names','IUCN_link'])
    driver = webdriver.Chrome(executable_path='./chromedriver');
    dz = pd.read_excel("assets/DataZone/HBW-BirdLife_List_of_Birds_v6.xlsx")
    list_of_birds = list(dz.loc[:,'Common name'])
    ds = 'None-error'
    try:
        f = int(input('from :'))
        t = int(input('to :'))
        ds = 'iucn_'+str(f)+'_'+str(t)
        logging.basicConfig(filename="assets/rescrape_iucn/Log_"+ds+".log",format='%(asctime)s - %(levelname)s - %(message)s',filemode='a')
        birds = iucn(f,t,driver,birds,list_of_birds)
    finally:
        birds.to_pickle("assets/rescrape_iucn/Pick_"+ds+".pkl")
        birds.to_csv("assets/rescrape_iucn/csv_"+ds+".csv")
        try:
            logging.info('\n\nTime taken : ',datetime.now() - startTime)
        finally:
            if len(fail) != 0 :
                print('falied birds : ',fail)
            print('\n\nTime taken : ',datetime.now() - startTime) 
            driver.quit()    
'''
conda activate indicwiki & cd Documents\birds & python Iucn_Scraper_Optim.py
<f>
<t>
'''


'''
1 
2 
3 
4 
5 ['Pangani Longclaw', 'Pangani Longclaw:Expandfail', 'Five-striped Sparrow', 'Five-striped Sparrow:Expandfail', 'Stripe-capped Sparrow', 'Stripe-capped Sparrow:Expandfail', 'Chestnut-capped Brush-finch', 'Chestnut-capped Brush-finch:Expandfail', 'Amazonian Oropendola', 'Amazonian Oropendola:Expandfail', 'Greater Antillean Grackle', 'Greater Antillean Grackle:Expandfail', 'Hermit Warbler', 'Hermit Warbler:Expandfail', 'Scarlet-and-white Tanager', 'Scarlet-and-white Tanager:Expandfail', 'Slate-colored Seedeater', 'Slate-colored Seedeater:Expandfail', 'Buff-bellied Tanager', 'Buff-bellied Tanager:Expandfail', 'Rusty-browed Warbling-finch', 'Rusty-browed Warbling-finch:Expandfail', 'Three-striped Hemispingus', 'Three-striped Hemispingus:Expandfail', 'Ochraceous Conebill', 'Ochraceous Conebill:Expandfail', 'Rusty Flowerpiercer', 'Rusty Flowerpiercer:Expandfail', 'Scaled Flowerpiercer', 'Scaled Flowerpiercer:Expandfail', 'Black-throated Flowerpiercer', 'Black-throated Flowerpiercer:Expandfail', 'Merida Flowerpiercer', 'Merida Flowerpiercer:Expandfail', 'Grass-green Tanager', 'Grass-green Tanager:Expandfail', 'Black-capped Tanager', 'Black-capped Tanager:Expandfail']
'''

In [3]:
from tkinter import N
from typing import final
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd
import csv
import requests
import time
from datetime import datetime
import logging
from timeit import default_timer as timer
import numpy as np
import pandas as pd
from time import sleep


In [4]:

driver = webdriver.Chrome(executable_path='./chromedriver');
birdname = 'American Crow'
driver.get('https://www.wikidata.org/wiki/Wikidata:Main_Page')
input_wd = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[placeholder='Search Wikidata']")))
input_wd.send_keys(birdname)
input_wd.send_keys(Keys.ENTER)

C:\Users\LEGION\AppData\Local\Temp\ipykernel_21140\3897690117.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver');


In [10]:
flag  = False
confirm = '//*[@id="mw-content-text"]/div[4]/ul/li[1]/div[2]/span'
sea = driver.find_elements(by=By.XPATH, value=confirm)[0].text.strip()
if sea == 'species of bird':
    flag = True

In [11]:
if flag:
    first_res ='//*[@id="mw-content-text"]/div[4]/ul/li[1]/div[1]/a'
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, first_res)))
    driver.find_elements(by=By.XPATH, value=first_res)[0].click()
    

In [12]:
dz = pd.read_pickle("data/Birds.pkl")
list_of_birds = list(dz.loc[:,'Bird Original Name'])


In [14]:
len(list_of_birds)

12658

In [15]:
driver.close()


In [3]:
!pip install scrapy

  Using cached Scrapy-2.6.1-py2.py3-none-any.whl (264 kB)
  Using cached Protego-0.2.1-py2.py3-none-any.whl (8.2 kB)
  Using cached PyDispatcher-2.0.5-py3-none-any.whl
  Using cached queuelib-1.6.2-py2.py3-none-any.whl (13 kB)
  Using cached itemadapter-0.5.0-py3-none-any.whl (10 kB)
  Using cached service_identity-21.1.0-py2.py3-none-any.whl (12 kB)
  Using cached cssselect-1.1.0-py2.py3-none-any.whl (16 kB)
  Using cached w3lib-1.22.0-py2.py3-none-any.whl (20 kB)
  Using cached zope.interface-5.4.0-cp39-cp39-win_amd64.whl (210 kB)
  Using cached itemloaders-1.0.4-py3-none-any.whl (11 kB)
  Using cached parsel-1.6.0-py2.py3-none-any.whl (13 kB)
  Using cached jmespath-1.0.0-py3-none-any.whl (23 kB)
  Using cached Automat-20.2.0-py2.py3-none-any.whl (31 kB)
  Using cached incremental-21.3.0-py2.py3-none-any.whl (15 kB)
  Using cached twisted_iocpsupport-1.0.2-cp39-cp39-win_amd64.whl (45 kB)
  Using cached constantly-15.1.0-py2.py3-none-any.whl (7.9 kB)
  Using cached hyperlink-21.0.0-p

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import scrapy
import pandas as pd
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import os
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from lxml import etree

In [ ]:

def scrape_v_3_ebird(URL):
    if URL != None :
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find_all("div",class_="Breadcrumbs")
        names_main = soup.find_all("span",class_="Heading-main")
        names_sub = soup.find_all("span",class_="Heading-sub")
        udescription=soup.find_all("p",class_="u-stack-sm")
        # print(birds)
        for i,j,k,l in zip(results,names_main,names_sub,udescription):
            a = i.get_text().strip().split('\n')
            b = j.get_text().strip()
            c = k.get_text().strip()
            e = l.get_text()
            d = {'Common Name':[b],'Kingdom':['Animalia'],'Phylum':['Chordata'],'Class':['Aves'],'Order':[a[0]],'Family':[a[1]],'Binomial Name':[c],'Description':[e],'Link':[URL]}
            return pd.DataFrame(d) 
    emp_d = {'Common Name':[b],'Kingdom':pd.notna,'Phylum':pd.notna,'Class':pd.notna,'Order':pd.notna,'Family':pd.notna,'Binomial Name':pd.notna,'Description':pd.notna,'Link':[URL]} 
    
    return pd.DataFrame(emp_d)
    

In [9]:
Url = 'https://www.wikidata.org/wiki/Q839854'
page = requests.get(Url)
soup = BeautifulSoup(page.content, "html.parser")

In [10]:
cs_xp = '//*[@id="q1923584$719E3AD2-C229-4A30-B8B2-93ECEAEDE2EA"]/div[2]/div[1]/div/div[2]/div[2]/div[1]/a'
//*[@id="q839854$0F0B42B0-3766-4703-9689-4F5891A28254"]/div[2]/div[1]/div/div[2]/div[2]/div[1]/a
img_xp = '//*[@id="q839854$F18A1C35-0744-465C-A2BD-AC082E429BFB"]/div[2]/div[1]/div/div[2]/div[2]/div[1]/div[2]/a'
dom = etree.HTML(str(soup))
print(dom.xpath(cs_xp)[0].text)
print(dom.xpath(img_xp)[0].text)

IndexError: list index out of range

In [11]:
!pip install wikidata

In [12]:
from wikidata.client import Client
client = Client()

In [14]:
Url = 'https://www.wikidata.org/wiki/Q839854'
entity = client.get(Url.split('/')[-1], load=True)

In [15]:
entity

<wikidata.entity.Entity Q839854 "Abbott's Booby">

In [16]:
entity.description

m'species of bird'

In [17]:
image_prop = client.get('P18')

In [18]:
image = entity[image_prop]

In [24]:
imagename =''
for i in str(image).split('"'):
    if 'File:' in i:
        imagename = i.split(':')[-1]
imagename

"Abbott's Booby.jpg"

In [20]:
image.image_resolution

(1018, 948)

In [21]:
image.image_url

'https://upload.wikimedia.org/wikipedia/commons/5/53/Abbott%27s_Booby.jpg'

In [25]:
image.title

"File:Abbott's Booby.jpg"

In [26]:
wikilink = pd.read_csv('assets\Wikidata_Ultimate_Source\links.csv')

In [31]:
wikilink.head(4)

,Unnamed: 0,Name,Wikidata_link,image
0,0,Abbott's Booby,https://www.wikidata.org/wiki/Q839854,NaN
1,0,Abbott's Starling,https://www.wikidata.org/wiki/Q27075615,NaN
2,0,Abd al Kuri Sparrow,https://www.wikidata.org/wiki/Q3056159,NaN
3,0,Abdim's Stork,https://www.wikidata.org/wiki/Q308199,NaN


In [44]:
import numpy as np 
wikilink['image'] = np.nan

In [48]:
def get_img(link):
    if not pd.isna(link):
        try:
            entity = client.get(link.split('/')[-1], load=True)
            image_prop = client.get('P18')
            image = entity[image_prop]
            t = image.title
            if 'File:' in t:
                imagename = t.strip().split(':')[-1]
                return imagename
            return np.nan
        except:
            return np.nan
    return np.nan
# get_img('https://www.wikidata.org/wiki/Q8061335')

In [49]:
from tqdm import tqdm
wikilink.reset_index(inplace=True)

In [50]:
for i in tqdm(range(wikilink.shape[0])):
    wikilink.loc[i,'image'] = get_img(wikilink.loc[i,'Wikidata_link'])

 44%|████▍     | 5540/12657 [4:28:01<2:48:51,  1.42s/it]     

Error: Canceled future for execute_request message before replies were done

In [ ]:
wikilink.to_pickle('assets\Wikidata_Ultimate_Source\link_image.pickle')

In [ ]:
wikilink.to_csv('assets\Wikidata_Ultimate_Source\link_image.csv')
wikilink.to_excel('assets\Wikidata_Ultimate_Source\link_image.excel')

In [3]:
images = pd.read_pickle('assets/Wikidata_Ultimate_Source/assets/Wikidata_Ultimate_Source/link_image.pkl')

In [4]:
images.head(5)

,index,Unnamed: 0,Name,Wikidata_link,image
0,0,0,Abbott's Booby,https://www.wikidata.org/wiki/Q839854,Abbott's Booby.jpg
1,1,0,Abbott's Starling,https://www.wikidata.org/wiki/Q27075615,Abbott's Starling (adult).jpg
2,2,0,Abd al Kuri Sparrow,https://www.wikidata.org/wiki/Q3056159,Passer hemileucus.jpg
3,3,0,Abdim's Stork,https://www.wikidata.org/wiki/Q308199,Ciconia abdimii -London Zoo-8a.jpg
4,4,0,Aberdare Cisticola,https://www.wikidata.org/wiki/Q1923584,NaN


In [9]:
images.drop(columns=['index','Unnamed: 0'],inplace =True)
images.describe()

,Name,Wikidata_link,image
count,12657,10197,8925
unique,12657,9822,8562
top,Abbott's Booby,https://www.wikidata.org/wiki/Q21416306,"Myiagra ferrocyanea ferrocyanea, Myiagra cervi..."
freq,1,3,3


In [10]:
birds = pd.read_pickle('data\Birds.pkl')
birds.head(5)

,Bird Original Name,Bird Original Name Telugu,Scientific Name,Scientific Name Telugu,Synonyms,Synonyms Telugu,Old latin name for bird,Old latin name for bird Telugu,Scientist,Scientist Telugu,...,Wiki_Description,Wiki_Description Telugu,Ebird_Link,Ebird_Description,Ebird_Description Telugu,Eol_Link,Eol_Description,Eol_Description Telugu,Dibird_Link,IUCN_link
0,Abbott's Babbler,అబోట్ యొక్క బాబ్లెర్,Malacocincla abbotti,మాలాకోసిన్లా అబ్బోటి,Abbott's Babbler,అబోట్ యొక్క బాబ్లెర్,NaN,NaN,Blyth,బ్లిత్,...,Abbott's babbler (Malacocincla abbotti) is a s...,అబోట్ యొక్క బాబ్లర్ (మలాకోసింక్లా అబోట్టి) అనే...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.iucnredlist.org/species/22715811/9...
1,Abbott's Booby,అబోట్ యొక్క బూబీ,Papasula abbotti,పాపసులా అబ్బోటి,Abbott's Booby,అబోట్ యొక్క బూబీ,Sula abbotti,సుల అబ్బోటి,Ridgway,రిడ్వే,...,Abbott's booby (Papasula abbotti) is an endang...,అబోట్ యొక్క బూబీ (పాపాసుల అబోట్టి) సులిడ్ కుటు...,https://ebird.org/species/abbboo2,"Gangly, stretched-out-looking black-and-white ...","గ్యాంగ్లీ, సాగదీసిన నలుపు-తెలుపు సముద్రపు పక్ష...",https://eol.org/pages/45513856,Papasula (Abbott's Booby) is a genus of birds ...,పాపసుల (అబాట్స్ బూబీ) అనేది సులిడే కుటుంబానికి...,https://dibird.com/species/abbotts-booby/,https://www.iucnredlist.org/species/22696649/1...
2,Abbott's Starling,అబోట్ యొక్క స్టార్లింగ్,Poeoptera femoralis,పూపెట్రా ఫెటోరిలిస్,Abbott's Starling,అబోట్ యొక్క స్టార్లింగ్,"Pholia femoralis, Pholidauges femoralis, Cinny...","ఫొలియా ఫెమోరిలిస్, ఫోలిడాగ్స్ ఫెటోరిలిస్, సినీ...",Richmond,రిచ్ మండ్,...,The Abbott's starling (Poeoptera femoralis) is...,అబోట్ యొక్క స్టార్లింగ్ (పోయోప్టెరా ఫెమోరాలిస్...,https://ebird.org/species/abbsta2,"A small, distinctive starling with a tiny rang...",కెన్యా మరియు టాంజానియాలో ఒక చిన్న శ్రేణితో ఒక ...,https://eol.org/pages/45515541,Poeoptera femoralis (Abbott's Starling) is a s...,పోయోప్టెరా ఫెమోరాలిస్ (అబాట్ యొక్క స్టార్లింగ్...,https://dibird.com/species/abbotts-starling/,https://www.iucnredlist.org/species/22710787/1...
3,Abd al Kuri Sparrow,అబ్ద్ అల్ కురి స్పారో,Passer hemileucus,పాసర్ హెమిలేయుస్,"Abd al Kuri Sparrow, Abd Al Kuri Sparrow","అబ్ద్ అల్ కురి స్పారో, అబ్ద్ అల్ కురి స్పారో",NaN,NaN,OgilvieGrant & Forbes,ఓగిల్వీగ్రాంట్ & ఫోర్బ్స్,...,Abd al Kuri (Arabic: عبد الكوري‎) is a rocky i...,అబ్ద్ అల్ కురి (అరబిక్: عبد الكوري) అనేది గార్...,NaN,NaN,NaN,NaN,NaN,NaN,https://dibird.com/species/abd-alkuri-sparrow/,https://www.iucnredlist.org/species/22735599/9...
4,Abdim's Stork,అబ్దిమ్ యొక్క స్టోర్క్,Ciconia abdimii,సికోనియా అబ్దిమిఐ,Abdim's Stork,అబ్దిమ్ యొక్క స్టోర్క్,"Sphenorrhynchus abdimii, Sphenorhynchus abdimii","స్ఫెనోరిన్చుస్ అబ్డిమి, స్ఫెన్నోరిచ్యున్చుస్ అ...",Lichtenstein,లిచ్టెన్ స్టెయిన్,...,"Abdim's stork (Ciconia abdimii), also known as...","అబ్దిమ్ కొంగ (సికోనియా అబ్దిమి), తెల్ల బొడ్డు ...",https://ebird.org/species/abdsto1,"A medium-sized, dark stork with a white belly....","తెల్లటి బొడ్డుతో మధ్యస్థ-పరిమాణ, ముదురు కొంగ. ...",https://eol.org/pages/45515774,Ciconia abdimii (Abdim's Stork) is a species o...,సికోనియా అబ్దిమి (అబ్డిమ్ కొంగ) అనేది సికోనిడే...,https://dibird.com/species/abdims-stork/,https://www.iucnredlist.org/species/22697673/9...


In [12]:
data = pd.merge(birds,images,left_on='Bird Original Name',right_on='Name')

In [14]:
row_dict = {}
print(list(data.columns))

['Bird Original Name', 'Bird Original Name Telugu', 'Scientific Name', 'Scientific Name Telugu', 'Synonyms', 'Synonyms Telugu', 'Old latin name for bird', 'Old latin name for bird Telugu', 'Scientist', 'Scientist Telugu', 'founded', 'founded Telugu', 'Kingdom', 'Kingdom Telugu', 'Subkingdom', 'Subkingdom Telugu', 'Infrakingdom', 'Infrakingdom Telugu', 'Phylum', 'Phylum Telugu', 'Subphylum', 'Subphylum Telugu', 'Infraphylum', 'Infraphylum Telugu', 'Superclass', 'Superclass Telugu', 'Class', 'Class Telugu', 'Order', 'Order Telugu', 'Family', 'Family Telugu', 'Genus', 'Genus Telugu', 'Species', 'Species Telugu', 'Conservation status', 'Conservation status Telugu', 'Habitat', 'Habitat Telugu', 'Country', 'Country Telugu', 'geographic distribution includes', 'geographic distribution includes Telugu', 'biogeographic realm', 'biogeographic realm Telugu', 'Population Growth', 'Population Growth Telugu', 'Breeding region', 'Breeding region Telugu', 'Breeding subregion', 'Breeding subregion Telu

In [15]:
temp = data[['image','Image']]
temp.describe()

,image,Image
count,8925,10523
unique,8562,6696
top,"Myiagra ferrocyanea ferrocyanea, Myiagra cervi...",https://upload.wikimedia.org/wikipedia/commons...
freq,3,995


In [16]:
data = data.drop(columns=['Image','References','Wiki_Description','Name'])

In [19]:
data

,Bird Original Name,Bird Original Name Telugu,Scientific Name,Scientific Name Telugu,Synonyms,Synonyms Telugu,Old latin name for bird,Old latin name for bird Telugu,Scientist,Scientist Telugu,...,Ebird_Link,Ebird_Description,Ebird_Description Telugu,Eol_Link,Eol_Description,Eol_Description Telugu,Dibird_Link,IUCN_link,Wikidata_link,image
0,Abbott's Booby,అబోట్ యొక్క బూబీ,Papasula abbotti,పాపసులా అబ్బోటి,Abbott's Booby,అబోట్ యొక్క బూబీ,Sula abbotti,సుల అబ్బోటి,Ridgway,రిడ్వే,...,https://ebird.org/species/abbboo2,"Gangly, stretched-out-looking black-and-white ...","గ్యాంగ్లీ, సాగదీసిన నలుపు-తెలుపు సముద్రపు పక్ష...",https://eol.org/pages/45513856,Papasula (Abbott's Booby) is a genus of birds ...,పాపసుల (అబాట్స్ బూబీ) అనేది సులిడే కుటుంబానికి...,https://dibird.com/species/abbotts-booby/,https://www.iucnredlist.org/species/22696649/1...,https://www.wikidata.org/wiki/Q839854,Abbott's Booby.jpg
1,Abbott's Starling,అబోట్ యొక్క స్టార్లింగ్,Poeoptera femoralis,పూపెట్రా ఫెటోరిలిస్,Abbott's Starling,అబోట్ యొక్క స్టార్లింగ్,"Pholia femoralis, Pholidauges femoralis, Cinny...","ఫొలియా ఫెమోరిలిస్, ఫోలిడాగ్స్ ఫెటోరిలిస్, సినీ...",Richmond,రిచ్ మండ్,...,https://ebird.org/species/abbsta2,"A small, distinctive starling with a tiny rang...",కెన్యా మరియు టాంజానియాలో ఒక చిన్న శ్రేణితో ఒక ...,https://eol.org/pages/45515541,Poeoptera femoralis (Abbott's Starling) is a s...,పోయోప్టెరా ఫెమోరాలిస్ (అబాట్ యొక్క స్టార్లింగ్...,https://dibird.com/species/abbotts-starling/,https://www.iucnredlist.org/species/22710787/1...,https://www.wikidata.org/wiki/Q27075615,Abbott's Starling (adult).jpg
2,Abd al Kuri Sparrow,అబ్ద్ అల్ కురి స్పారో,Passer hemileucus,పాసర్ హెమిలేయుస్,"Abd al Kuri Sparrow, Abd Al Kuri Sparrow","అబ్ద్ అల్ కురి స్పారో, అబ్ద్ అల్ కురి స్పారో",NaN,NaN,OgilvieGrant & Forbes,ఓగిల్వీగ్రాంట్ & ఫోర్బ్స్,...,NaN,NaN,NaN,NaN,NaN,NaN,https://dibird.com/species/abd-alkuri-sparrow/,https://www.iucnredlist.org/species/22735599/9...,https://www.wikidata.org/wiki/Q3056159,Passer hemileucus.jpg
3,Abdim's Stork,అబ్దిమ్ యొక్క స్టోర్క్,Ciconia abdimii,సికోనియా అబ్దిమిఐ,Abdim's Stork,అబ్దిమ్ యొక్క స్టోర్క్,"Sphenorrhynchus abdimii, Sphenorhynchus abdimii","స్ఫెనోరిన్చుస్ అబ్డిమి, స్ఫెన్నోరిచ్యున్చుస్ అ...",Lichtenstein,లిచ్టెన్ స్టెయిన్,...,https://ebird.org/species/abdsto1,"A medium-sized, dark stork with a white belly....","తెల్లటి బొడ్డుతో మధ్యస్థ-పరిమాణ, ముదురు కొంగ. ...",https://eol.org/pages/45515774,Ciconia abdimii (Abdim's Stork) is a species o...,సికోనియా అబ్దిమి (అబ్డిమ్ కొంగ) అనేది సికోనిడే...,https://dibird.com/species/abdims-stork/,https://www.iucnredlist.org/species/22697673/9...,https://www.wikidata.org/wiki/Q308199,Ciconia abdimii -London Zoo-8a.jpg
4,Aberdare Cisticola,అబెర్డేర్ సిస్టోలా,Cisticola aberdare,సిస్కోలా అబెర్డేర్,Aberdare Cisticola,అబెర్డేర్ సిస్టోలా,Cisticola robusta aberdare,సిస్కోలా రొస్టా అబెర్డేర్,Lynes,లయన్స్,...,https://ebird.org/species/abecis1,A large and relatively colorful cisticola ende...,కెన్యాకు చెందిన పెద్ద మరియు సాపేక్షంగా రంగురంగ...,https://eol.org/pages/45514495,Cisticola aberdare (Aberdare Cisticola) is a s...,సిస్టికోలా అబెర్‌డేర్ (అబెర్‌డేర్ సిస్టికోలా) ...,https://dibird.com/species/aberdare-cisticola/,https://www.iucnredlist.org/species/22713436/1...,https://www.wikidata.org/wiki/Q1923584,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12652,Yucatan Rough-winged Swallow,యుకాటన్ రఫ్-వింగ్ స్వో,Stelgidopteryx ridgwayi,స్టెల్గిడోపోటెక్స్ రిడ్జ్వేయి,"Yucatan Rough-winged Swallow, Ridgway's Rough-...","యుకాటన్ రఫ్-రెక్కల స్వో, రిడ్వేయా యొక్క రఫ్-రె...",NaN,NaN,Nelson,నెల్సన్,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.iucnredlist.org/species/22712157/1...,NaN,NaN
12653,Yunnan Parrotbill,యున్నాన్ పార్ట్ బిల్,Suthora ricketti,సుథోరా రికెట్టి,Yunnan Parrotbill,యున్నాన్ పార్ట్ బిల్,NaN,NaN,Rothschild,రోత్స్కిల్డ్,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.iucnredlist.org/species/103887001/...,https://www.wikidata.org/wiki/Q8061335,NaN
12654,Zambian Barbet,జాంబియన్ బార్బెట్,Ly

In [20]:
data[['Wikidata_link']]

,Wikidata_link
0,https://www.wikidata.org/wiki/Q839854
1,https://www.wikidata.org/wiki/Q27075615
2,https://www.wikidata.org/wiki/Q3056159
3,https://www.wikidata.org/wiki/Q308199
4,https://www.wikidata.org/wiki/Q1923584
...,...
12652,NaN
12653,https://www.wikidata.org/wiki/Q8061335
12654,https://www.wikidata.org/wiki/Q1261209
12655,https://www.wikidata.org/wiki/Q31874195


In [34]:
cs_reclean = pd.read_csv('Week1_Strucured_Data_Gen\kaggleout\Dataset_final_v3_3.csv')

In [35]:
cs_reclean['Conservation status']

0        Least Concern
1           Endangered
2           Endangered
3           Vulnerable
4        Least Concern
             ...      
12653    Least Concern
12654    Least Concern
12655       Vulnerable
12656       Vulnerable
12657    Least Concern
Name: Conservation status, Length: 12658, dtype: object

In [39]:
import re
import numpy as np
import string
def clean_cs(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    if type(text) == str and text.lower().strip() not in ('Not evaluated'.lower(),'Data Deficient'.lower(),'',' ','cites appendix ii','data deficient','range restricted'):
        text = text.lower()
        text = re.sub("[\(\[].*?[\)\]]", "", text)
        text = re.sub('\n', '', text)
        # text = re.sub('\[.*?\]', '', text)
        # text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub('[-‘’“”…]', '', text)
        # text = re.sub(' ', '', text)
        text = re.sub('[0-9]', '', text)
        if 'least concern' in text or 'lc' in text:
            return 'least concern'
        if 'wild' in text:
            return 'Extinct in the Wild'
        if 'extinct' in text:
            return 'Extinct'
        if 'appendix' in text:
            return np.nan
        return text.strip()
    return np.nan
clean_c = lambda x: clean_cs(x)
cs_reclean.rename({'Conservation status':'cs'},axis=1,inplace=True)
cs_reclean['Conservation status'] = cs_reclean.cs.apply(clean_c)
cs_reclean['Conservation status'] = cs_reclean.cs.apply(clean_c)
cs_reclean.drop(['cs'],axis=1,inplace=True)
k =[]
for i in list(cs_reclean['Conservation status']) :
    if type(i) != float:
        k.append(i)
set(k)

{'Extinct',
 'Extinct in the Wild',
 'critically endangered',
 'endangered',
 'least concern',
 'near threatened',
 'vulnerable'}

In [48]:
seven_tel = {'Extinct':'అంతరించిపోయింది','Extinct in the Wild':'అడవిలో అంతరించిపోయింది','critically endangered':'ప్రమాదస్థాయిలో  అంతరించిపోతున్న జాతులు','endangered':'అంతరించిపోతున్న జాతులు','vulnerable':'భవిషత్తులో అంతరించిపోయే అవకాశం వున్నా జాతులు','near threatened':'సమీప ముప్పు','least concern':'తక్కువ ఆందోళన జాతులు',np.nan:np.nan}

In [55]:
seven_cs = {'Extinct':'EX','Extinct in the Wild':'EW','critically endangered':'CR','endangered':'EN','vulnerable':'VU','near threatened':'NT','least concern':'LC',np.nan:np.nan}

In [41]:
data['Conservation status'] = cs_reclean['Conservation status']

In [45]:
data['Conservation status']

0        least concern
1           endangered
2           endangered
3           vulnerable
4        least concern
             ...      
12652    least concern
12653    least concern
12654    least concern
12655       vulnerable
12656       vulnerable
Name: Conservation status, Length: 12657, dtype: object

In [50]:
list2 = []
for i in list(cs_reclean['Conservation status']):
    list2.append(seven_tel[i])
new_tel_cs = pd.Series(list2)

In [56]:
code_list2 = []
for i in list(cs_reclean['Conservation status']):
    code_list2.append(seven_cs[i])
new_tel_cs_code = pd.Series(code_list2)

In [57]:
data['Conservation status code'] = new_tel_cs_code

In [71]:
data.head(5)

,Bird Original Name,Bird Original Name Telugu,Scientific Name,Scientific Name Telugu,Synonyms,Synonyms Telugu,Old latin name for bird,Old latin name for bird Telugu,Scientist,Scientist Telugu,founded,founded Telugu,Kingdom,Kingdom Telugu,Subkingdom,Subkingdom Telugu,Infrakingdom,Infrakingdom Telugu,Phylum,Phylum Telugu,Subphylum,Subphylum Telugu,Infraphylum,Infraphylum Telugu,Superclass,Superclass Telugu,Class,Class Telugu,Order,Order Telugu,Family,Family Telugu,Genus,Genus Telugu,Species,Species Telugu,Conservation status,Conservation status Telugu,Habitat,Habitat Telugu,Country,Country Telugu,geographic distribution includes,geographic distribution includes Telugu,biogeographic realm,biogeographic realm Telugu,Population Growth,Population Growth Telugu,Breeding region,Breeding region Telugu,Breeding subregion,Breeding subregion Telugu,Life Span,Life Span Telugu,altitude,altitude Telugu,visual system,visual system Telugu,auditory system,auditory system Telugu,hearing range,hearing range Telugu,hearing threshold,hearing threshold Telugu,locomotion,locomotion Telugu,mineralized skeleton contains,mineralized skeleton contains Telugu,body mass,body mass Telugu,Wiki_Link,Wiki_Description Telugu,Ebird_Link,Ebird_Description,Ebird_Description Telugu,Eol_Link,Eol_Description,Eol_Description Telugu,Dibird_Link,IUCN_link,Wikidata_link,image,Conservation status code,Page Id
0,Abbott's Booby,అబోట్ యొక్క బూబీ,Papasula abbotti,పాపసులా అబ్బోటి,Abbott's Booby,అబోట్ యొక్క బూబీ,Sula abbotti,సుల అబ్బోటి,Ridgway,రిడ్వే,1893.0,1893.0,Animalia,జంతువులు,Bilateria,బిలేటేరియా,Deuterostomia,డ్యూటెరోస్టోమియా,Chordata,చోర్డేటా,Vertebrata,వెన్నుపూస,Gnathostomata,గ్నాథోస్టోమాటా,Tetrapoda,టెట్రాపోడా,Aves,ఏవ్స్ (పక్షులు),Suliformes,సులిఫార్మ్స్,Sulidae,సులిదా,"PapasulaOlson & Warheit, 1988",పాపసులా,P. abbotti,పి. అబ్బోటి,least concern,తక్కువ ఆందోళన జాతులు,"Forest, Marine Neritic, Marine Oceanic","అటవీ, మెరైన్ నెరిటిక్, మెరైన్ ఓసియానిక్",Christmas Island,క్రిస్మస్ ద్వీపం,NaN,NaN,NaN,NaN,Stable,స్థిరమైన,Indian ocean,భారతీయ మహాసముద్రం,northeast,ఈశాన్య,14.1 years,"13,1 సంవత్సరాలు",260 metres,260 మీటర్లు,corneal eyes,కార్నియా కళ్ళు,tympanic middle ear,టిమ్పానిక్ మధ్య చెవి,5000 Hz,5000 హెర్ట్జ్(Hz),20 dB,20 డెసిబెల్స్ (dB),flight,ఎగరగల,apatite,అపటైట్,NaN,NaN,https://en.wikipedia.org/wiki/Abbott%27s_booby,అబోట్ యొక్క బూబీ (పాపాసుల అబోట్టి) సులిడ్ కుటు...,https://ebird.org/species/abbboo2,"Gangly, stretched-out-looking black-and-white ...","గ్యాంగ్లీ, సాగదీసిన నలుపు-తెలుపు సముద్రపు పక్ష...",https://eol.org/pages/45513856,Papasula (Abbott's Booby) is a genus of birds ...,పాపసుల (అబాట్స్ బూబీ) అనేది సులిడే కుటుంబానికి...,https://dibird.com/species/abbotts-booby/,https://www.iucnredlist.org/species/22696649/1...,https://www.wikidata.org/wiki/Q839854,Abbott's Booby.jpg,LC,T22696649A152726109
1,Abbott's Starling,అబోట్ యొక్క స్టార్లింగ్,Poeoptera femoralis,పూపెట్రా ఫెటోరిలిస్,Abbott's Starling,అబోట్ యొక్క స్టార్లింగ్,"Pholia femoralis, Pholidauges femoralis, Cinny...","ఫొలియా ఫెమోరిలిస్, ఫోలిడాగ్స్ ఫెటోరిలిస్, సినీ...",Richmond,రిచ్ మండ్,1897.0,1897.0,Animalia,జంతువులు,Bilateria,బిలేటేరియా,Deuterostomia,డ్యూటెరోస్టోమియా,Chordata,చోర్డేటా,Vertebrata,వెన్నుపూస,Gnathostomata,గ్నాథోస్టోమాటా,Tetrapoda,టెట్రాపోడా,Aves,ఏవ్స్ (పక్షులు),Passeriformes,పాస్రిఫార్మ్స్,Sturnidae,స్టర్నిడైడ్,Poeoptera,పూపెట్రా,P. femoralis,పి. ఫెటోరిలిస్,endangered,అంతరించిపోతున్న జాతులు,"tropical,terrestrial","ఉష్ణమండల, భూమధ్యరేఖ",Kenya,కెన్యా,Tanzania,టాంజానియా,NaN,NaN,Decreasing,తగ్గుతోంది,Africa,ఆఫ్రికా,Kenya and Tanzania,కెన్యా మరియు టాంజానియా.,3.5 years,3.5 సంవత్సరాలు,"2,600 metres","2,600 మీటర్లు",corneal eyes,కార్నియా కళ్ళు,tympanic middle ear,టిమ్పానిక్ మధ్య చెవి,5000 Hz,5000 హెర్ట్జ్(Hz),10 dB,10 డెసిబెల్స్ (dB),bipedal movement,ద్విపాద కదలిక,apatite,అపటైట్,NaN,NaN,https://en.wikipedia.org/wiki/Abbott%27s_starling,అబోట్ యొక్క స్టార్లింగ్ (పోయోప్టెరా ఫెమోరాలిస్...,https://ebird.org/species/abbsta2,"A small, distinctive starling with a tiny rang...",కెన్యా మరియు టాంజానియాలో ఒక చిన్న శ్రేణితో ఒ

In [72]:
row_dict = {}
print(list(data.columns))

['Bird Original Name', 'Bird Original Name Telugu', 'Scientific Name', 'Scientific Name Telugu', 'Synonyms', 'Synonyms Telugu', 'Old latin name for bird', 'Old latin name for bird Telugu', 'Scientist', 'Scientist Telugu', 'founded', 'founded Telugu', 'Kingdom', 'Kingdom Telugu', 'Subkingdom', 'Subkingdom Telugu', 'Infrakingdom', 'Infrakingdom Telugu', 'Phylum', 'Phylum Telugu', 'Subphylum', 'Subphylum Telugu', 'Infraphylum', 'Infraphylum Telugu', 'Superclass', 'Superclass Telugu', 'Class', 'Class Telugu', 'Order', 'Order Telugu', 'Family', 'Family Telugu', 'Genus', 'Genus Telugu', 'Species', 'Species Telugu', 'Conservation status', 'Conservation status Telugu', 'Habitat', 'Habitat Telugu', 'Country', 'Country Telugu', 'geographic distribution includes', 'geographic distribution includes Telugu', 'biogeographic realm', 'biogeographic realm Telugu', 'Population Growth', 'Population Growth Telugu', 'Breeding region', 'Breeding region Telugu', 'Breeding subregion', 'Breeding subregion Telu

In [73]:
cols = ['Bird Original Name', 'Bird Original Name Telugu', 'Scientific Name', 'Scientific Name Telugu', 'Synonyms', 'Synonyms Telugu', 'Old latin name for bird', 'Old latin name for bird Telugu', 'Scientist', 'Scientist Telugu', 'founded', 'founded Telugu', 'Kingdom', 'Kingdom Telugu', 'Subkingdom', 'Subkingdom Telugu', 'Infrakingdom', 'Infrakingdom Telugu', 'Phylum', 'Phylum Telugu', 'Subphylum', 'Subphylum Telugu', 'Infraphylum', 'Infraphylum Telugu', 'Superclass', 'Superclass Telugu', 'Class', 'Class Telugu', 'Order', 'Order Telugu', 'Family', 'Family Telugu', 'Genus', 'Genus Telugu', 'Species', 'Species Telugu', 'Conservation status', 'Conservation status Telugu', 'Habitat', 'Habitat Telugu', 'Country', 'Country Telugu', 'geographic distribution includes', 'geographic distribution includes Telugu', 'biogeographic realm', 'biogeographic realm Telugu', 'Population Growth', 'Population Growth Telugu', 'Breeding region', 'Breeding region Telugu', 'Breeding subregion', 'Breeding subregion Telugu', 'Life Span', 'Life Span Telugu', 'altitude', 'altitude Telugu', 'visual system', 'visual system Telugu', 'auditory system', 'auditory system Telugu', 'hearing range', 'hearing range Telugu', 'hearing threshold', 'hearing threshold Telugu', 'locomotion', 'locomotion Telugu', 'mineralized skeleton contains', 'mineralized skeleton contains Telugu', 'body mass', 'body mass Telugu', 'Wiki_Link', 'Wiki_Description Telugu', 'Ebird_Link', 'Ebird_Description', 'Ebird_Description Telugu', 'Eol_Link', 'Eol_Description', 'Eol_Description Telugu', 'Dibird_Link', 'IUCN_link', 'Wikidata_link', 'image', 'Conservation status code','Page Id']
len(cols)

84

In [74]:
for i in cols:
    row_dict['_'.join(i.split(' '))] = ''
row_dict

{'Bird_Original_Name': '',
 'Bird_Original_Name_Telugu': '',
 'Scientific_Name': '',
 'Scientific_Name_Telugu': '',
 'Synonyms': '',
 'Synonyms_Telugu': '',
 'Old_latin_name_for_bird': '',
 'Old_latin_name_for_bird_Telugu': '',
 'Scientist': '',
 'Scientist_Telugu': '',
 'founded': '',
 'founded_Telugu': '',
 'Kingdom': '',
 'Kingdom_Telugu': '',
 'Subkingdom': '',
 'Subkingdom_Telugu': '',
 'Infrakingdom': '',
 'Infrakingdom_Telugu': '',
 'Phylum': '',
 'Phylum_Telugu': '',
 'Subphylum': '',
 'Subphylum_Telugu': '',
 'Infraphylum': '',
 'Infraphylum_Telugu': '',
 'Superclass': '',
 'Superclass_Telugu': '',
 'Class': '',
 'Class_Telugu': '',
 'Order': '',
 'Order_Telugu': '',
 'Family': '',
 'Family_Telugu': '',
 'Genus': '',
 'Genus_Telugu': '',
 'Species': '',
 'Species_Telugu': '',
 'Conservation_status': '',
 'Conservation_status_Telugu': '',
 'Habitat': '',
 'Habitat_Telugu': '',
 'Country': '',
 'Country_Telugu': '',
 'geographic_distribution_includes': '',
 'geographic_distribut

In [70]:

page = []
for i in list(data['IUCN_link']):
    if type(i) != float:
        p = i.strip().split('/')
        pid = 'T' + p[-2] + 'A' + p[-1]
        page.append(pid)
    else:
        page.append(np.nan)
data['Page Id'] = page

In [18]:
import pickle
import ast
from jinja2 import Environment, FileSystemLoader,Template
import string
from hashlib import sha1
from datetime import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm
import jinja2
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [75]:
# b = []
# for i in row_dict.keys():
#     b.append(i)
# print(b)

['Bird_Original_Name', 'Bird_Original_Name_Telugu', 'Scientific_Name', 'Scientific_Name_Telugu', 'Synonyms', 'Synonyms_Telugu', 'Old_latin_name_for_bird', 'Old_latin_name_for_bird_Telugu', 'Scientist', 'Scientist_Telugu', 'founded', 'founded_Telugu', 'Kingdom', 'Kingdom_Telugu', 'Subkingdom', 'Subkingdom_Telugu', 'Infrakingdom', 'Infrakingdom_Telugu', 'Phylum', 'Phylum_Telugu', 'Subphylum', 'Subphylum_Telugu', 'Infraphylum', 'Infraphylum_Telugu', 'Superclass', 'Superclass_Telugu', 'Class', 'Class_Telugu', 'Order', 'Order_Telugu', 'Family', 'Family_Telugu', 'Genus', 'Genus_Telugu', 'Species', 'Species_Telugu', 'Conservation_status', 'Conservation_status_Telugu', 'Habitat', 'Habitat_Telugu', 'Country', 'Country_Telugu', 'geographic_distribution_includes', 'geographic_distribution_includes_Telugu', 'biogeographic_realm', 'biogeographic_realm_Telugu', 'Population_Growth', 'Population_Growth_Telugu', 'Breeding_region', 'Breeding_region_Telugu', 'Breeding_subregion', 'Breeding_subregion_Telu

In [89]:
# T = """ {{Speciesbox | name = {{Bird_Original_Name_Telugu}} {%- if Conservation_status_code!= 'NaN' %} | status = {{Conservation_status_code}} {%- endif %} | status_system = IUCN3.1 {%- if Page_Id!= 'NaN' %} | status_ref=<ref name="iucn status 18 November 2021">{{cite iucn |author=BirdLife International |date=2016 |title=''{{Bird_Original_Name}}'' |volume=2016 |page=e.{{Page_Id}} |doi=10.2305/IUCN.UK.2016-3.RLTS.{{Page_Id}}.en |access-date=18 November 2021}}</ref> {%- endif %} {%- if image!= 'NaN' %} | image = {{image}} {%- endif %} | regnum= జంతువులు | phylum= కార్డేట్ | classis=[[పక్షులు|ఏవ్స్]]  {%- if Order_Telugu!= 'NaN' %} | ordo={{Order_Telugu}} {%- endif %}{%- if Family_Telugu!= 'NaN' %} | familia={{Family_Telugu}} {%- endif %} {%- if Genus_Telugu!= 'NaN' %} | genus =''{{Genus_Telugu}}'' {%- endif %} {%- if Species_Telugu!= 'NaN' %} | species = '''''{{Species_Telugu}}''''' {%- endif %} {%- if Scientist_Telugu!= 'NaN' and founded_Telugu != 'NaN' %} | authority = ([[{{Scientist_Telugu}}]], {{founded_Telugu}}) {%- endif %} {%- if Synonyms_Telugu!= 'NaN' %} | synonyms = {{Synonyms_Telugu}} {%- endif %} }} """  

In [19]:
file_loader = FileSystemLoader('./Template/')
env = Environment(loader=file_loader)
template = env.get_template('birds_infobox.j2')

In [76]:
# data.to_csv('data/data_for_template.csv')

In [18]:
B = "Abd al Kuri Sparrow"

In [16]:
import pandas as pd
data = pd.read_csv('data\data_for_template.csv')

In [22]:
data.drop(columns=['Unnamed: 0'],inplace=True)
data.set_index('Bird Original Name',inplace=True)
data.head(4)

,Bird Original Name Telugu,Scientific Name,Scientific Name Telugu,Synonyms,Synonyms Telugu,Old latin name for bird,Old latin name for bird Telugu,Scientist,Scientist Telugu,founded,founded Telugu,Kingdom,Kingdom Telugu,Subkingdom,Subkingdom Telugu,Infrakingdom,Infrakingdom Telugu,Phylum,Phylum Telugu,Subphylum,Subphylum Telugu,Infraphylum,Infraphylum Telugu,Superclass,Superclass Telugu,Class,Class Telugu,Order,Order Telugu,Family,Family Telugu,Genus,Genus Telugu,Species,Species Telugu,Conservation status,Conservation status Telugu,Habitat,Habitat Telugu,Country,Country Telugu,geographic distribution includes,geographic distribution includes Telugu,biogeographic realm,biogeographic realm Telugu,Population Growth,Population Growth Telugu,Breeding region,Breeding region Telugu,Breeding subregion,Breeding subregion Telugu,Life Span,Life Span Telugu,altitude,altitude Telugu,visual system,visual system Telugu,auditory system,auditory system Telugu,hearing range,hearing range Telugu,hearing threshold,hearing threshold Telugu,locomotion,locomotion Telugu,mineralized skeleton contains,mineralized skeleton contains Telugu,body mass,body mass Telugu,Wiki_Link,Wiki_Description Telugu,Ebird_Link,Ebird_Description,Ebird_Description Telugu,Eol_Link,Eol_Description,Eol_Description Telugu,Dibird_Link,IUCN_link,Wikidata_link,image,Conservation status code,Page Id
Bird Original Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abbott's Booby,అబోట్ యొక్క బూబీ,Papasula abbotti,పాపసులా అబ్బోటి,Abbott's Booby,అబోట్ యొక్క బూబీ,Sula abbotti,సుల అబ్బోటి,Ridgway,రిడ్వే,1893.0,1893.0,Animalia,జంతువులు,Bilateria,బిలేటేరియా,Deuterostomia,డ్యూటెరోస్టోమియా,Chordata,చోర్డేటా,Vertebrata,వెన్నుపూస,Gnathostomata,గ్నాథోస్టోమాటా,Tetrapoda,టెట్రాపోడా,Aves,ఏవ్స్ (పక్షులు),Suliformes,సులిఫార్మ్స్,Sulidae,సులిదా,"PapasulaOlson & Warheit, 1988",పాపసులా,P. abbotti,పి. అబ్బోటి,least concern,తక్కువ ఆందోళన జాతులు,"Forest, Marine Neritic, Marine Oceanic","అటవీ, మెరైన్ నెరిటిక్, మెరైన్ ఓసియానిక్",Christmas Island,క్రిస్మస్ ద్వీపం,NaN,NaN,NaN,NaN,Stable,స్థిరమైన,Indian ocean,భారతీయ మహాసముద్రం,northeast,ఈశాన్య,14.1 years,"13,1 సంవత్సరాలు",260 metres,260 మీటర్లు,corneal eyes,కార్నియా కళ్ళు,tympanic middle ear,టిమ్పానిక్ మధ్య చెవి,5000 Hz,5000 హెర్ట్జ్(Hz),20 dB,20 డెసిబెల్స్ (dB),flight,ఎగరగల,apatite,అపటైట్,NaN,NaN,https://en.wikipedia.org/wiki/Abbott%27s_booby,అబోట్ యొక్క బూబీ (పాపాసుల అబోట్టి) సులిడ్ కుటు...,https://ebird.org/species/abbboo2,"Gangly, stretched-out-looking black-and-white ...","గ్యాంగ్లీ, సాగదీసిన నలుపు-తెలుపు సముద్రపు పక్ష...",https://eol.org/pages/45513856,Papasula (Abbott's Booby) is a genus of birds ...,పాపసుల (అబాట్స్ బూబీ) అనేది సులిడే కుటుంబానికి...,https://dibird.com/species/abbotts-booby/,https://www.iucnredlist.org/species/22696649/1...,https://www.wikidata.org/wiki/Q839854,Abbott's Booby.jpg,LC,T22696649A152726109
Abbott's Starling,అబోట్ యొక్క స్టార్లింగ్,Poeoptera femoralis,పూపెట్రా ఫెటోరిలిస్,Abbott's Starling,అబోట్ యొక్క స్టార్లింగ్,"Pholia femoralis, Pholidauges femoralis, Cinny...","ఫొలియా ఫెమోరిలిస్, ఫోలిడాగ్స్ ఫెటోరిలిస్, సినీ...",Richmond,రిచ్ మండ్,1897.0,1897.0,Animalia,జంతువులు,Bilateria,బిలేటేరియా,Deuterostomia,డ్యూటెరోస్టోమియా,Chordata,చోర్డేటా,Vertebrata,వెన్నుపూస,Gnathostomata,గ్నాథోస్టోమాటా,Tetrapoda,టెట్రాపోడా,Aves,ఏవ్స్ (పక్షులు),Passeriformes,పాస్రిఫార్మ్స్,Sturnidae,స్టర్నిడైడ్,Poeoptera,పూపెట్రా,P. femoralis,పి. ఫెటోరిలిస్,endangered,అంతరించిపోతున్న జాతులు,"tropical,terrestrial","ఉష్ణమండల, భూమధ్యరేఖ",Kenya,కెన్యా,Tanzania,టాంజానియా,NaN,NaN,Decreasing,తగ్గుతోంది,Africa,ఆఫ్రికా,Kenya and Tanzania,కెన్యా మరియు టాంజానియా.,3.5 years,3.5 సంవత్సరాలు,"2,600 metres","2,600 మీటర్లు",corneal eyes,కార్నియా కళ్ళు,tympanic middle ear,టిమ్పానిక్ మధ్య చెవి,5000 Hz,5000 హెర్ట్జ్(Hz),10 dB,10 డెసిబెల్స్ (dB),bipedal movement,ద్విపాద కదలిక,apatite,అపటైట్,NaN,NaN,https://en.wikipedia.org/wiki/Abbott%27s_starling,అబోట్ యొక్క స్టార్లింగ్ (పోయోప్టెరా ఫెమోరాలిస్...,https://ebird.org/species/abbsta2,"A small, distinc

In [23]:
# data.loc[B]

Bird Original Name Telugu                                              అబ్ద్ అల్ కురి స్పారో
Scientific Name                                                            Passer hemileucus
Scientific Name Telugu                                                      పాసర్ హెమిలేయుస్
Synonyms                                            Abd al Kuri Sparrow, Abd Al Kuri Sparrow
Synonyms Telugu                                 అబ్ద్ అల్ కురి స్పారో, అబ్ద్ అల్ కురి స్పారో
Old latin name for bird                                                                  NaN
Old latin name for bird Telugu                                                           NaN
Scientist                                                              OgilvieGrant & Forbes
Scientist Telugu                                                   ఓగిల్వీగ్రాంట్ & ఫోర్బ్స్
founded                                                                               1900.0
founded Telugu                                                        

In [118]:
# data['founded Telugu'] = data['founded Telugu'].astype(int)
# data.head(4)

,Bird Original Name Telugu,Scientific Name,Scientific Name Telugu,Synonyms,Synonyms Telugu,Old latin name for bird,Old latin name for bird Telugu,Scientist,Scientist Telugu,founded,founded Telugu,Kingdom,Kingdom Telugu,Subkingdom,Subkingdom Telugu,Infrakingdom,Infrakingdom Telugu,Phylum,Phylum Telugu,Subphylum,Subphylum Telugu,Infraphylum,Infraphylum Telugu,Superclass,Superclass Telugu,Class,Class Telugu,Order,Order Telugu,Family,Family Telugu,Genus,Genus Telugu,Species,Species Telugu,Conservation status,Conservation status Telugu,Habitat,Habitat Telugu,Country,Country Telugu,geographic distribution includes,geographic distribution includes Telugu,biogeographic realm,biogeographic realm Telugu,Population Growth,Population Growth Telugu,Breeding region,Breeding region Telugu,Breeding subregion,Breeding subregion Telugu,Life Span,Life Span Telugu,altitude,altitude Telugu,visual system,visual system Telugu,auditory system,auditory system Telugu,hearing range,hearing range Telugu,hearing threshold,hearing threshold Telugu,locomotion,locomotion Telugu,mineralized skeleton contains,mineralized skeleton contains Telugu,body mass,body mass Telugu,Wiki_Link,Wiki_Description Telugu,Ebird_Link,Ebird_Description,Ebird_Description Telugu,Eol_Link,Eol_Description,Eol_Description Telugu,Dibird_Link,IUCN_link,Wikidata_link,image,Conservation status code,Page Id
Bird Original Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abbott's Booby,అబోట్ యొక్క బూబీ,Papasula abbotti,పాపసులా అబ్బోటి,Abbott's Booby,అబోట్ యొక్క బూబీ,Sula abbotti,సుల అబ్బోటి,Ridgway,రిడ్వే,1893.0,1893.0,Animalia,జంతువులు,Bilateria,బిలేటేరియా,Deuterostomia,డ్యూటెరోస్టోమియా,Chordata,చోర్డేటా,Vertebrata,వెన్నుపూస,Gnathostomata,గ్నాథోస్టోమాటా,Tetrapoda,టెట్రాపోడా,Aves,ఏవ్స్ (పక్షులు),Suliformes,సులిఫార్మ్స్,Sulidae,సులిదా,"PapasulaOlson & Warheit, 1988",పాపసులా,P. abbotti,పి. అబ్బోటి,least concern,తక్కువ ఆందోళన జాతులు,"Forest, Marine Neritic, Marine Oceanic","అటవీ, మెరైన్ నెరిటిక్, మెరైన్ ఓసియానిక్",Christmas Island,క్రిస్మస్ ద్వీపం,NaN,NaN,NaN,NaN,Stable,స్థిరమైన,Indian ocean,భారతీయ మహాసముద్రం,northeast,ఈశాన్య,14.1 years,"13,1 సంవత్సరాలు",260 metres,260 మీటర్లు,corneal eyes,కార్నియా కళ్ళు,tympanic middle ear,టిమ్పానిక్ మధ్య చెవి,5000 Hz,5000 హెర్ట్జ్(Hz),20 dB,20 డెసిబెల్స్ (dB),flight,ఎగరగల,apatite,అపటైట్,NaN,NaN,https://en.wikipedia.org/wiki/Abbott%27s_booby,అబోట్ యొక్క బూబీ (పాపాసుల అబోట్టి) సులిడ్ కుటు...,https://ebird.org/species/abbboo2,"Gangly, stretched-out-looking black-and-white ...","గ్యాంగ్లీ, సాగదీసిన నలుపు-తెలుపు సముద్రపు పక్ష...",https://eol.org/pages/45513856,Papasula (Abbott's Booby) is a genus of birds ...,పాపసుల (అబాట్స్ బూబీ) అనేది సులిడే కుటుంబానికి...,https://dibird.com/species/abbotts-booby/,https://www.iucnredlist.org/species/22696649/1...,https://www.wikidata.org/wiki/Q839854,Abbott's Booby.jpg,LC,T22696649A152726109
Abbott's Starling,అబోట్ యొక్క స్టార్లింగ్,Poeoptera femoralis,పూపెట్రా ఫెటోరిలిస్,Abbott's Starling,అబోట్ యొక్క స్టార్లింగ్,"Pholia femoralis, Pholidauges femoralis, Cinny...","ఫొలియా ఫెమోరిలిస్, ఫోలిడాగ్స్ ఫెటోరిలిస్, సినీ...",Richmond,రిచ్ మండ్,1897.0,1897.0,Animalia,జంతువులు,Bilateria,బిలేటేరియా,Deuterostomia,డ్యూటెరోస్టోమియా,Chordata,చోర్డేటా,Vertebrata,వెన్నుపూస,Gnathostomata,గ్నాథోస్టోమాటా,Tetrapoda,టెట్రాపోడా,Aves,ఏవ్స్ (పక్షులు),Passeriformes,పాస్రిఫార్మ్స్,Sturnidae,స్టర్నిడైడ్,Poeoptera,పూపెట్రా,P. femoralis,పి. ఫెటోరిలిస్,endangered,అంతరించిపోతున్న జాతులు,"tropical,terrestrial","ఉష్ణమండల, భూమధ్యరేఖ",Kenya,కెన్యా,Tanzania,టాంజానియా,NaN,NaN,Decreasing,తగ్గుతోంది,Africa,ఆఫ్రికా,Kenya and Tanzania,కెన్యా మరియు టాంజానియా.,3.5 years,3.5 సంవత్సరాలు,"2,600 metres","2,600 మీటర్లు",corneal eyes,కార్నియా కళ్ళు,tympanic middle ear,టిమ్పానిక్ మధ్య చెవి,5000 Hz,5000 హెర్ట్జ్(Hz),10 dB,10 డెసిబెల్స్ (dB),bipedal movement,ద్విపాద కదలిక,apatite,అపటైట్,NaN,NaN,https://en.wikipedia.org/wiki/Abbott%27s_starling,అబోట్ యొక్క స్టార్లింగ్ (పోయోప్టెరా ఫెమోరాలిస్...,https://ebird.org/species/abbsta2,"A small, distinc

In [24]:
# data.set_index("Bird Original Name",inplace=True)
row_dict = {}
bird_dict = dict(data.loc[B])
for k,v in zip(bird_dict,bird_dict.values()):
    if not pd.isna(v):
        row_dict["_".join(k.split(' '))] = v
    else:
        row_dict["_".join(k.split(' '))] = 'NaN'

In [25]:
print(template.render(row_dict))

{{Speciesbox
| name= అబ్ద్ అల్ కురి స్పారో
| status = EN
| status_system = IUCN3.1
| status_ref=<ref name="iucn status 18 November 2021">{{cite iucn |author=BirdLife International |date=2016 |title='''' |volume=2016 |page=e.T22735599A95115321 |doi=10.2305/IUCN.UK.2016-3.RLTS.T22735599A95115321.en |access-date=18 November 2021}}</ref>
| image = Passer hemileucus.jpg
| regnum= జంతువులు
| phylum= కార్డేట్
| classis=[[పక్షులు|ఏవ్స్]]
| ordo=పాస్రిఫార్మ్స్
| familia=ప్యాసెరిడా
| genus =''పాసర్''
| species = '''''పి. హెమిలేయుస్'''''
| authority = ([[ఓగిల్వీగ్రాంట్ & ఫోర్బ్స్]], 1900)
| synonyms = అబ్ద్ అల్ కురి స్పారో, అబ్ద్ అల్ కురి స్పారో
}}


In [5]:
# data.head(5)

,Unnamed: 0,Bird Original Name,Bird Original Name Telugu,Scientific Name,Scientific Name Telugu,Synonyms,Synonyms Telugu,Old latin name for bird,Old latin name for bird Telugu,Scientist,Scientist Telugu,founded,founded Telugu,Kingdom,Kingdom Telugu,Subkingdom,Subkingdom Telugu,Infrakingdom,Infrakingdom Telugu,Phylum,Phylum Telugu,Subphylum,Subphylum Telugu,Infraphylum,Infraphylum Telugu,Superclass,Superclass Telugu,Class,Class Telugu,Order,Order Telugu,Family,Family Telugu,Genus,Genus Telugu,Species,Species Telugu,Conservation status,Conservation status Telugu,Habitat,Habitat Telugu,Country,Country Telugu,geographic distribution includes,geographic distribution includes Telugu,biogeographic realm,biogeographic realm Telugu,Population Growth,Population Growth Telugu,Breeding region,Breeding region Telugu,Breeding subregion,Breeding subregion Telugu,Life Span,Life Span Telugu,altitude,altitude Telugu,visual system,visual system Telugu,auditory system,auditory system Telugu,hearing range,hearing range Telugu,hearing threshold,hearing threshold Telugu,locomotion,locomotion Telugu,mineralized skeleton contains,mineralized skeleton contains Telugu,body mass,body mass Telugu,Wiki_Link,Wiki_Description Telugu,Ebird_Link,Ebird_Description,Ebird_Description Telugu,Eol_Link,Eol_Description,Eol_Description Telugu,Dibird_Link,IUCN_link,Wikidata_link,image,Conservation status code,Page Id
0,0,Abbott's Booby,అబోట్ యొక్క బూబీ,Papasula abbotti,పాపసులా అబ్బోటి,Abbott's Booby,అబోట్ యొక్క బూబీ,Sula abbotti,సుల అబ్బోటి,Ridgway,రిడ్వే,1893.0,1893.0,Animalia,జంతువులు,Bilateria,బిలేటేరియా,Deuterostomia,డ్యూటెరోస్టోమియా,Chordata,చోర్డేటా,Vertebrata,వెన్నుపూస,Gnathostomata,గ్నాథోస్టోమాటా,Tetrapoda,టెట్రాపోడా,Aves,ఏవ్స్ (పక్షులు),Suliformes,సులిఫార్మ్స్,Sulidae,సులిదా,"PapasulaOlson & Warheit, 1988",పాపసులా,P. abbotti,పి. అబ్బోటి,least concern,తక్కువ ఆందోళన జాతులు,"Forest, Marine Neritic, Marine Oceanic","అటవీ, మెరైన్ నెరిటిక్, మెరైన్ ఓసియానిక్",Christmas Island,క్రిస్మస్ ద్వీపం,NaN,NaN,NaN,NaN,Stable,స్థిరమైన,Indian ocean,భారతీయ మహాసముద్రం,northeast,ఈశాన్య,14.1 years,"13,1 సంవత్సరాలు",260 metres,260 మీటర్లు,corneal eyes,కార్నియా కళ్ళు,tympanic middle ear,టిమ్పానిక్ మధ్య చెవి,5000 Hz,5000 హెర్ట్జ్(Hz),20 dB,20 డెసిబెల్స్ (dB),flight,ఎగరగల,apatite,అపటైట్,NaN,NaN,https://en.wikipedia.org/wiki/Abbott%27s_booby,అబోట్ యొక్క బూబీ (పాపాసుల అబోట్టి) సులిడ్ కుటు...,https://ebird.org/species/abbboo2,"Gangly, stretched-out-looking black-and-white ...","గ్యాంగ్లీ, సాగదీసిన నలుపు-తెలుపు సముద్రపు పక్ష...",https://eol.org/pages/45513856,Papasula (Abbott's Booby) is a genus of birds ...,పాపసుల (అబాట్స్ బూబీ) అనేది సులిడే కుటుంబానికి...,https://dibird.com/species/abbotts-booby/,https://www.iucnredlist.org/species/22696649/1...,https://www.wikidata.org/wiki/Q839854,Abbott's Booby.jpg,LC,T22696649A152726109
1,1,Abbott's Starling,అబోట్ యొక్క స్టార్లింగ్,Poeoptera femoralis,పూపెట్రా ఫెటోరిలిస్,Abbott's Starling,అబోట్ యొక్క స్టార్లింగ్,"Pholia femoralis, Pholidauges femoralis, Cinny...","ఫొలియా ఫెమోరిలిస్, ఫోలిడాగ్స్ ఫెటోరిలిస్, సినీ...",Richmond,రిచ్ మండ్,1897.0,1897.0,Animalia,జంతువులు,Bilateria,బిలేటేరియా,Deuterostomia,డ్యూటెరోస్టోమియా,Chordata,చోర్డేటా,Vertebrata,వెన్నుపూస,Gnathostomata,గ్నాథోస్టోమాటా,Tetrapoda,టెట్రాపోడా,Aves,ఏవ్స్ (పక్షులు),Passeriformes,పాస్రిఫార్మ్స్,Sturnidae,స్టర్నిడైడ్,Poeoptera,పూపెట్రా,P. femoralis,పి. ఫెటోరిలిస్,endangered,అంతరించిపోతున్న జాతులు,"tropical,terrestrial","ఉష్ణమండల, భూమధ్యరేఖ",Kenya,కెన్యా,Tanzania,టాంజానియా,NaN,NaN,Decreasing,తగ్గుతోంది,Africa,ఆఫ్రికా,Kenya and Tanzania,కెన్యా మరియు టాంజానియా.,3.5 years,3.5 సంవత్సరాలు,"2,600 metres","2,600 మీటర్లు",corneal eyes,కార్నియా కళ్ళు,tympanic middle ear,టిమ్పానిక్ మధ్య చెవి,5000 Hz,5000 హెర్ట్జ్(Hz),10 dB,10 డెసిబెల్స్ (dB),bipedal movement,ద్విపాద కదలిక,apatite,అపటైట్,NaN,NaN,https://en.wikipedia.org/wiki/Abbott%27s_starling,అబోట్ యొక్క స్టార్లింగ్ (పోయోప్టెరా ఫెమోరాలిస్...,https://ebird.org/species/abbsta2,"A small, distinctive starling with a tiny rang...",కెన్యా మరియు టాంజానియాలో ఒక చ

In [6]:
# col_t = ['Bird Original Name', 'Bird Original Name Telugu', 'Scientific Name', 'Scientific Name Telugu', 'Synonyms', 'Synonyms Telugu', 'Old latin name for bird', 'Old latin name for bird Telugu', 'Scientist', 'Scientist Telugu', 'founded', 'founded Telugu', 'Kingdom', 'Kingdom Telugu', 'Subkingdom', 'Subkingdom Telugu', 'Infrakingdom', 'Infrakingdom Telugu', 'Phylum', 'Phylum Telugu', 'Subphylum', 'Subphylum Telugu', 'Infraphylum', 'Infraphylum Telugu', 'Superclass', 'Superclass Telugu', 'Class', 'Class Telugu', 'Order', 'Order Telugu', 'Family', 'Family Telugu', 'Genus', 'Genus Telugu', 'Species', 'Species Telugu', 'Conservation status', 'Conservation status Telugu', 'Habitat', 'Habitat Telugu', 'Country', 'Country Telugu', 'geographic distribution includes', 'geographic distribution includes Telugu', 'biogeographic realm', 'biogeographic realm Telugu', 'Population Growth', 'Population Growth Telugu', 'Breeding region', 'Breeding region Telugu', 'Breeding subregion', 'Breeding subregion Telugu', 'Life Span', 'Life Span Telugu', 'altitude', 'altitude Telugu', 'visual system', 'visual system Telugu', 'auditory system', 'auditory system Telugu', 'hearing range', 'hearing range Telugu', 'hearing threshold', 'hearing threshold Telugu', 'locomotion', 'locomotion Telugu', 'mineralized skeleton contains', 'mineralized skeleton contains Telugu', 'body mass', 'body mass Telugu', 'Wiki_Link', 'image', 'Wiki_Description', 'Wiki_Description Telugu', 'Ebird_Link', 'Ebird_Description', 'Ebird_Description Telugu', 'Eol_Link', 'Eol_Description', 'Eol_Description Telugu', 'Dibird_Link', 'IUCN_link']

In [7]:
# for i in list(data.columns):
#     print(i)

Unnamed: 0
Bird Original Name
Bird Original Name Telugu
Scientific Name
Scientific Name Telugu
Synonyms
Synonyms Telugu
Old latin name for bird
Old latin name for bird Telugu
Scientist
Scientist Telugu
founded
founded Telugu
Kingdom
Kingdom Telugu
Subkingdom
Subkingdom Telugu
Infrakingdom
Infrakingdom Telugu
Phylum
Phylum Telugu
Subphylum
Subphylum Telugu
Infraphylum
Infraphylum Telugu
Superclass
Superclass Telugu
Class
Class Telugu
Order
Order Telugu
Family
Family Telugu
Genus
Genus Telugu
Species
Species Telugu
Conservation status
Conservation status Telugu
Habitat
Habitat Telugu
Country
Country Telugu
geographic distribution includes
geographic distribution includes Telugu
biogeographic realm
biogeographic realm Telugu
Population Growth
Population Growth Telugu
Breeding region
Breeding region Telugu
Breeding subregion
Breeding subregion Telugu
Life Span
Life Span Telugu
altitude
altitude Telugu
visual system
visual system Telugu
auditory system
auditory system Telugu
hearing range
h

In [12]:
# data.tail()

,Unnamed: 0,Bird Original Name,Bird Original Name Telugu,Scientific Name,Scientific Name Telugu,Synonyms,Synonyms Telugu,Old latin name for bird,Old latin name for bird Telugu,Scientist,Scientist Telugu,founded,founded Telugu,Kingdom,Kingdom Telugu,Subkingdom,Subkingdom Telugu,Infrakingdom,Infrakingdom Telugu,Phylum,Phylum Telugu,Subphylum,Subphylum Telugu,Infraphylum,Infraphylum Telugu,Superclass,Superclass Telugu,Class,Class Telugu,Order,Order Telugu,Family,Family Telugu,Genus,Genus Telugu,Species,Species Telugu,Conservation status,Conservation status Telugu,Habitat,Habitat Telugu,Country,Country Telugu,geographic distribution includes,geographic distribution includes Telugu,biogeographic realm,biogeographic realm Telugu,Population Growth,Population Growth Telugu,Breeding region,Breeding region Telugu,Breeding subregion,Breeding subregion Telugu,Life Span,Life Span Telugu,altitude,altitude Telugu,visual system,visual system Telugu,auditory system,auditory system Telugu,hearing range,hearing range Telugu,hearing threshold,hearing threshold Telugu,locomotion,locomotion Telugu,mineralized skeleton contains,mineralized skeleton contains Telugu,body mass,body mass Telugu,Wiki_Link,Wiki_Description Telugu,Ebird_Link,Ebird_Description,Ebird_Description Telugu,Eol_Link,Eol_Description,Eol_Description Telugu,Dibird_Link,IUCN_link,Wikidata_link,image,Conservation status code,Page Id
12652,12652,Yucatan Rough-winged Swallow,యుకాటన్ రఫ్-వింగ్ స్వో,Stelgidopteryx ridgwayi,స్టెల్గిడోపోటెక్స్ రిడ్జ్వేయి,"Yucatan Rough-winged Swallow, Ridgway's Rough-...","యుకాటన్ రఫ్-రెక్కల స్వో, రిడ్వేయా యొక్క రఫ్-రె...",NaN,NaN,Nelson,నెల్సన్,1901.0,1901.0,Animalia,జంతువులు,Bilateria,బిలేటేరియా,Deuterostomia,డ్యూటెరోస్టోమియా,Chordata,చోర్డేటా,Vertebrata,వెన్నుపూస,Gnathostomata,గ్నాథోస్టోమాటా,Tetrapoda,టెట్రాపోడా,Aves,ఏవ్స్ (పక్షులు),passeriformes,పాసేరిఫోడ్స్,hirundinidae,హరికొండీదై,stelgidopteryx,స్టెల్గిడోపోటెక్స్,NaN,NaN,least concern,తక్కువ ఆందోళన జాతులు,"Forest, Wetlands (inland), Rocky areas (eg. in...","అటవీ, వెట్ల్యాండ్స్ (భూభాగం), రాకీ ప్రాంతాలు (...",NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,3.3 years,3.3 సంవత్సరాలు,"2,500 metres","2,500 మీటర్లు",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.iucnredlist.org/species/22712157/1...,NaN,NaN,LC,T22712157A118843169
12653,12653,Yunnan Parrotbill,యున్నాన్ పార్ట్ బిల్,Suthora ricketti,సుథోరా రికెట్టి,Yunnan Parrotbill,యున్నాన్ పార్ట్ బిల్,NaN,NaN,Rothschild,రోత్స్కిల్డ్,1922.0,1922.0,Animalia,జంతువులు,Bilateria,బిలేటేరియా,Deuterostomia,డ్యూటెరోస్టోమియా,Chordata,చోర్డేటా,Vertebrata,వెన్నుపూస,Gnathostomata,గ్నాథోస్టోమాటా,Tetrapoda,టెట్రాపోడా,Aves,ఏవ్స్ (పక్షులు),passeriformes,పాసేరిఫోడ్స్,paradoxornithidae,పారడాక్సోనిథిడే,suthora,సుథోరా,NaN,NaN,least concern,తక్కువ ఆందోళన జాతులు,"Forest, Shrubland, Grassland","అటవీ, ష్రాబ్లాండ్, గ్రాస్ ల్యాండ్",NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,4.6 years,46 సంవత్సరాలు,"2,800 metres","2,800 మీటర్లు",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.iucnredlist.org/species/103887001/...,https://www.wikidata.org/wiki/Q8061335,NaN,LC,T103887001A113121204
12654,12654,Zambian Barbet,జాంబియన్ బార్బెట్,Lybius chaplini,లిబియస్ చాప్లిని,"Zambian Barbet, Chaplin's Barbet","జాంబియన్ బార్బెట్, చాప్లిన్ యొక్క బార్బెట్",NaN,NaN,Clarke,క్లార్క్,1920.0,1920.0,Animalia,జంతువులు,Bilateria,బిలేటేరియా,Deuterostomia,డ్యూటెరోస్టోమియా,Chordata,చోర్డేటా,Vertebrata,వెన్నుపూస,Gnathostomata,గ్నాథోస్టోమాటా,Tetrapoda,టెట్రాపోడా,Aves,ఏవ్స్ (పక్షులు),piciformes,పిసిఫోడ్స్,lybiidae,లిబిఇడా,lybius,లిబియస్,NaN,NaN,least concern,తక్కువ ఆందోళన జాతులు,"Savanna, Grassland, Artificial/Terrestrial","సవాన్నా, గ్రాస్ ల్యాండ్, ఆర్టిఫిషియల్ / టెర్రె...",NaN,NaN,NaN,NaN,NaN,NaN,Decreasing,తగ్గుతోంది,NaN,NaN,NaN,NaN,8.5 years,8.5 సంవత్సరాలు,"1,400 metres","1,400 మీటర్లు",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.iucnred

### Merge New Names ?

In [21]:
import pandas as pd
new = pd.read_pickle('data\Bird_Names.pkl')
old = pd.read_csv('data\data_for_template.csv')

In [22]:
old.shape[0]

12657

In [23]:
new.shape[0]

12658

In [24]:
for i in range(old.shape[0]):
    old.loc[i,'Bird Original Name Telugu'] = new.loc[i+1,'Name Translate'] 

In [25]:
old.tail(4)

,Unnamed: 0,Bird Original Name,Bird Original Name Telugu,Scientific Name,Scientific Name Telugu,Synonyms,Synonyms Telugu,Old latin name for bird,Old latin name for bird Telugu,Scientist,Scientist Telugu,founded,founded Telugu,Kingdom,Kingdom Telugu,Subkingdom,Subkingdom Telugu,Infrakingdom,Infrakingdom Telugu,Phylum,Phylum Telugu,Subphylum,Subphylum Telugu,Infraphylum,Infraphylum Telugu,Superclass,Superclass Telugu,Class,Class Telugu,Order,Order Telugu,Family,Family Telugu,Genus,Genus Telugu,Species,Species Telugu,Conservation status,Conservation status Telugu,Habitat,Habitat Telugu,Country,Country Telugu,geographic distribution includes,geographic distribution includes Telugu,biogeographic realm,biogeographic realm Telugu,Population Growth,Population Growth Telugu,Breeding region,Breeding region Telugu,Breeding subregion,Breeding subregion Telugu,Life Span,Life Span Telugu,altitude,altitude Telugu,visual system,visual system Telugu,auditory system,auditory system Telugu,hearing range,hearing range Telugu,hearing threshold,hearing threshold Telugu,locomotion,locomotion Telugu,mineralized skeleton contains,mineralized skeleton contains Telugu,body mass,body mass Telugu,Wiki_Link,Wiki_Description Telugu,Ebird_Link,Ebird_Description,Ebird_Description Telugu,Eol_Link,Eol_Description,Eol_Description Telugu,Dibird_Link,IUCN_link,Wikidata_link,image,Conservation status code,Page Id
12653,12653,Yunnan Parrotbill,యునాన్ పారోట్బిల్,Suthora ricketti,సుథోరా రికెట్టి,Yunnan Parrotbill,యున్నాన్ పార్ట్ బిల్,NaN,NaN,Rothschild,రోత్స్కిల్డ్,1922.0,1922.0,Animalia,జంతువులు,Bilateria,బిలేటేరియా,Deuterostomia,డ్యూటెరోస్టోమియా,Chordata,చోర్డేటా,Vertebrata,వెన్నుపూస,Gnathostomata,గ్నాథోస్టోమాటా,Tetrapoda,టెట్రాపోడా,Aves,ఏవ్స్ (పక్షులు),passeriformes,పాసేరిఫోడ్స్,paradoxornithidae,పారడాక్సోనిథిడే,suthora,సుథోరా,NaN,NaN,least concern,తక్కువ ఆందోళన జాతులు,"Forest, Shrubland, Grassland","అటవీ, ష్రాబ్లాండ్, గ్రాస్ ల్యాండ్",NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,4.6 years,46 సంవత్సరాలు,"2,800 metres","2,800 మీటర్లు",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.iucnredlist.org/species/103887001/...,https://www.wikidata.org/wiki/Q8061335,NaN,LC,T103887001A113121204
12654,12654,Zambian Barbet,జాంబియన్ బార్బెట్,Lybius chaplini,లిబియస్ చాప్లిని,"Zambian Barbet, Chaplin's Barbet","జాంబియన్ బార్బెట్, చాప్లిన్ యొక్క బార్బెట్",NaN,NaN,Clarke,క్లార్క్,1920.0,1920.0,Animalia,జంతువులు,Bilateria,బిలేటేరియా,Deuterostomia,డ్యూటెరోస్టోమియా,Chordata,చోర్డేటా,Vertebrata,వెన్నుపూస,Gnathostomata,గ్నాథోస్టోమాటా,Tetrapoda,టెట్రాపోడా,Aves,ఏవ్స్ (పక్షులు),piciformes,పిసిఫోడ్స్,lybiidae,లిబిఇడా,lybius,లిబియస్,NaN,NaN,least concern,తక్కువ ఆందోళన జాతులు,"Savanna, Grassland, Artificial/Terrestrial","సవాన్నా, గ్రాస్ ల్యాండ్, ఆర్టిఫిషియల్ / టెర్రె...",NaN,NaN,NaN,NaN,NaN,NaN,Decreasing,తగ్గుతోంది,NaN,NaN,NaN,NaN,8.5 years,8.5 సంవత్సరాలు,"1,400 metres","1,400 మీటర్లు",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.iucnredlist.org/species/22681832/9...,https://www.wikidata.org/wiki/Q1261209,Barbet-zambian.jpg,LC,T22681832A92922084
12655,12655,Zambian Yellow Warbler,జాంబియన్ ఎల్లో వార్బ్లెర్,Calamonastides bensoni,కాలోమోనాస్టిడ్స్ బెన్సోని,Zambian Yellow Warbler,జాంబియన్ ఎల్లో వార్బ్లెర్,NaN,NaN,Amadon,అమడన్,1954.0,1954.0,Animalia,జంతువులు,Bilateria,బిలేటేరియా,Deuterostomia,డ్యూటెరోస్టోమియా,Chordata,చోర్డేటా,Vertebrata,వెన్నుపూస,Gnathostomata,గ్నాథోస్టోమాటా,Tetrapoda,టెట్రాపోడా,Aves,ఏవ్స్ (పక్షులు),passeriformes,పాసేరిఫోడ్స్,acrocephalidae,ఎక్రోసెఫాలిడే,calamonastides,కాలోమోనాస్టిడ్స్,NaN,NaN,vulnerable,భవిషత్తులో అంతరించిపోయే అవకాశం వున్నా జాతులు,Wetlands (inland),తడిభూములు (భూభాగం),NaN,NaN,NaN,NaN,NaN,NaN,Decreasing,తగ్గుతోంది,NaN,NaN,NaN,NaN,3.6 years,3.6 సంవత్సరాలు,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.iucnredlist.org/species/103779336/...,https://www.wikidata.org/wiki/Q31874195,NaN,

In [26]:
import numpy as np
for i in range(old.shape[0]):
    if old.loc[i,'Bird Original Name'] == old.loc[i,'Old latin name for bird']:
        old.loc[i,'Old latin name for bird'] = np.nan
        old.loc[i,'Old latin name for bird Telugu'] = np.nan

In [27]:
old.tail(5)

,Unnamed: 0,Bird Original Name,Bird Original Name Telugu,Scientific Name,Scientific Name Telugu,Synonyms,Synonyms Telugu,Old latin name for bird,Old latin name for bird Telugu,Scientist,Scientist Telugu,founded,founded Telugu,Kingdom,Kingdom Telugu,Subkingdom,Subkingdom Telugu,Infrakingdom,Infrakingdom Telugu,Phylum,Phylum Telugu,Subphylum,Subphylum Telugu,Infraphylum,Infraphylum Telugu,Superclass,Superclass Telugu,Class,Class Telugu,Order,Order Telugu,Family,Family Telugu,Genus,Genus Telugu,Species,Species Telugu,Conservation status,Conservation status Telugu,Habitat,Habitat Telugu,Country,Country Telugu,geographic distribution includes,geographic distribution includes Telugu,biogeographic realm,biogeographic realm Telugu,Population Growth,Population Growth Telugu,Breeding region,Breeding region Telugu,Breeding subregion,Breeding subregion Telugu,Life Span,Life Span Telugu,altitude,altitude Telugu,visual system,visual system Telugu,auditory system,auditory system Telugu,hearing range,hearing range Telugu,hearing threshold,hearing threshold Telugu,locomotion,locomotion Telugu,mineralized skeleton contains,mineralized skeleton contains Telugu,body mass,body mass Telugu,Wiki_Link,Wiki_Description Telugu,Ebird_Link,Ebird_Description,Ebird_Description Telugu,Eol_Link,Eol_Description,Eol_Description Telugu,Dibird_Link,IUCN_link,Wikidata_link,image,Conservation status code,Page Id
12652,12652,Yucatan Rough-winged Swallow,యుకాటాన్ రఫ్-వింగ్డ్ స్వాలో,Stelgidopteryx ridgwayi,స్టెల్గిడోపోటెక్స్ రిడ్జ్వేయి,"Yucatan Rough-winged Swallow, Ridgway's Rough-...","యుకాటన్ రఫ్-రెక్కల స్వో, రిడ్వేయా యొక్క రఫ్-రె...",NaN,NaN,Nelson,నెల్సన్,1901.0,1901.0,Animalia,జంతువులు,Bilateria,బిలేటేరియా,Deuterostomia,డ్యూటెరోస్టోమియా,Chordata,చోర్డేటా,Vertebrata,వెన్నుపూస,Gnathostomata,గ్నాథోస్టోమాటా,Tetrapoda,టెట్రాపోడా,Aves,ఏవ్స్ (పక్షులు),passeriformes,పాసేరిఫోడ్స్,hirundinidae,హరికొండీదై,stelgidopteryx,స్టెల్గిడోపోటెక్స్,NaN,NaN,least concern,తక్కువ ఆందోళన జాతులు,"Forest, Wetlands (inland), Rocky areas (eg. in...","అటవీ, వెట్ల్యాండ్స్ (భూభాగం), రాకీ ప్రాంతాలు (...",NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,3.3 years,3.3 సంవత్సరాలు,"2,500 metres","2,500 మీటర్లు",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.iucnredlist.org/species/22712157/1...,NaN,NaN,LC,T22712157A118843169
12653,12653,Yunnan Parrotbill,యునాన్ పారోట్బిల్,Suthora ricketti,సుథోరా రికెట్టి,Yunnan Parrotbill,యున్నాన్ పార్ట్ బిల్,NaN,NaN,Rothschild,రోత్స్కిల్డ్,1922.0,1922.0,Animalia,జంతువులు,Bilateria,బిలేటేరియా,Deuterostomia,డ్యూటెరోస్టోమియా,Chordata,చోర్డేటా,Vertebrata,వెన్నుపూస,Gnathostomata,గ్నాథోస్టోమాటా,Tetrapoda,టెట్రాపోడా,Aves,ఏవ్స్ (పక్షులు),passeriformes,పాసేరిఫోడ్స్,paradoxornithidae,పారడాక్సోనిథిడే,suthora,సుథోరా,NaN,NaN,least concern,తక్కువ ఆందోళన జాతులు,"Forest, Shrubland, Grassland","అటవీ, ష్రాబ్లాండ్, గ్రాస్ ల్యాండ్",NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,4.6 years,46 సంవత్సరాలు,"2,800 metres","2,800 మీటర్లు",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.iucnredlist.org/species/103887001/...,https://www.wikidata.org/wiki/Q8061335,NaN,LC,T103887001A113121204
12654,12654,Zambian Barbet,జాంబియన్ బార్బెట్,Lybius chaplini,లిబియస్ చాప్లిని,"Zambian Barbet, Chaplin's Barbet","జాంబియన్ బార్బెట్, చాప్లిన్ యొక్క బార్బెట్",NaN,NaN,Clarke,క్లార్క్,1920.0,1920.0,Animalia,జంతువులు,Bilateria,బిలేటేరియా,Deuterostomia,డ్యూటెరోస్టోమియా,Chordata,చోర్డేటా,Vertebrata,వెన్నుపూస,Gnathostomata,గ్నాథోస్టోమాటా,Tetrapoda,టెట్రాపోడా,Aves,ఏవ్స్ (పక్షులు),piciformes,పిసిఫోడ్స్,lybiidae,లిబిఇడా,lybius,లిబియస్,NaN,NaN,least concern,తక్కువ ఆందోళన జాతులు,"Savanna, Grassland, Artificial/Terrestrial","సవాన్నా, గ్రాస్ ల్యాండ్, ఆర్టిఫిషియల్ / టెర్రె...",NaN,NaN,NaN,NaN,NaN,NaN,Decreasing,తగ్గుతోంది,NaN,NaN,NaN,NaN,8.5 years,8.5 సంవత్సరాలు,"1,400 metres","1,400 మీటర్లు",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.iucnr

In [20]:
print(list(old.columns))

['Unnamed: 0', 'Bird Original Name', 'Bird Original Name Telugu', 'Scientific Name', 'Scientific Name Telugu', 'Synonyms', 'Synonyms Telugu', 'Old latin name for bird', 'Old latin name for bird Telugu', 'Scientist', 'Scientist Telugu', 'founded', 'founded Telugu', 'Kingdom', 'Kingdom Telugu', 'Subkingdom', 'Subkingdom Telugu', 'Infrakingdom', 'Infrakingdom Telugu', 'Phylum', 'Phylum Telugu', 'Subphylum', 'Subphylum Telugu', 'Infraphylum', 'Infraphylum Telugu', 'Superclass', 'Superclass Telugu', 'Class', 'Class Telugu', 'Order', 'Order Telugu', 'Family', 'Family Telugu', 'Genus', 'Genus Telugu', 'Species', 'Species Telugu', 'Conservation status', 'Conservation status Telugu', 'Habitat', 'Habitat Telugu', 'Country', 'Country Telugu', 'geographic distribution includes', 'geographic distribution includes Telugu', 'biogeographic realm', 'biogeographic realm Telugu', 'Population Growth', 'Population Growth Telugu', 'Breeding region', 'Breeding region Telugu', 'Breeding subregion', 'Breeding 

In [30]:
final_col = [ 'Bird Original Name', 'Bird Original Name Telugu', 'Scientific Name', 'Scientific Name Telugu','Synonyms Telugu','Old latin name for bird Telugu', 'Scientist', 'Scientist Telugu', 'founded','Kingdom', 'Kingdom Telugu', 'Phylum', 'Phylum Telugu', 'Class', 'Class Telugu', 'Order', 'Order Telugu', 'Family', 'Family Telugu', 'Genus', 'Genus Telugu', 'Species', 'Species Telugu','Conservation status Telugu','Habitat Telugu','Country Telugu','geographic distribution includes Telugu','biogeographic realm Telugu','Population Growth Telugu','Breeding region Telugu', 'Breeding subregion Telugu','Life Span Telugu','altitude Telugu','visual system Telugu', 'auditory system Telugu', 'hearing range Telugu',  'hearing threshold Telugu', 'locomotion Telugu','mineralized skeleton contains Telugu','body mass Telugu', 'Wiki_Link', 'Wiki_Description Telugu', 'Ebird_Link', 'Ebird_Description', 'Ebird_Description Telugu', 'Eol_Link', 'Eol_Description', 'Eol_Description Telugu', 'Dibird_Link', 'IUCN_link', 'Wikidata_link', 'image', 'Conservation status code', 'Page Id']

In [31]:
data = old[final_col]
data.head(4)

,Bird Original Name,Bird Original Name Telugu,Scientific Name,Scientific Name Telugu,Synonyms Telugu,Old latin name for bird Telugu,Scientist,Scientist Telugu,founded,Kingdom,Kingdom Telugu,Phylum,Phylum Telugu,Class,Class Telugu,Order,Order Telugu,Family,Family Telugu,Genus,Genus Telugu,Species,Species Telugu,Conservation status Telugu,Habitat Telugu,Country Telugu,geographic distribution includes Telugu,biogeographic realm Telugu,Population Growth Telugu,Breeding region Telugu,Breeding subregion Telugu,Life Span Telugu,altitude Telugu,visual system Telugu,auditory system Telugu,hearing range Telugu,hearing threshold Telugu,locomotion Telugu,mineralized skeleton contains Telugu,body mass Telugu,Wiki_Link,Wiki_Description Telugu,Ebird_Link,Ebird_Description,Ebird_Description Telugu,Eol_Link,Eol_Description,Eol_Description Telugu,Dibird_Link,IUCN_link,Wikidata_link,image,Conservation status code,Page Id
0,Abbott's Booby,అబాట్ యొక్క బూబీ,Papasula abbotti,పాపసులా అబ్బోటి,అబోట్ యొక్క బూబీ,సుల అబ్బోటి,Ridgway,రిడ్వే,1893.0,Animalia,జంతువులు,Chordata,చోర్డేటా,Aves,ఏవ్స్ (పక్షులు),Suliformes,సులిఫార్మ్స్,Sulidae,సులిదా,"PapasulaOlson & Warheit, 1988",పాపసులా,P. abbotti,పి. అబ్బోటి,తక్కువ ఆందోళన జాతులు,"అటవీ, మెరైన్ నెరిటిక్, మెరైన్ ఓసియానిక్",క్రిస్మస్ ద్వీపం,NaN,NaN,స్థిరమైన,భారతీయ మహాసముద్రం,ఈశాన్య,"13,1 సంవత్సరాలు",260 మీటర్లు,కార్నియా కళ్ళు,టిమ్పానిక్ మధ్య చెవి,5000 హెర్ట్జ్(Hz),20 డెసిబెల్స్ (dB),ఎగరగల,అపటైట్,NaN,https://en.wikipedia.org/wiki/Abbott%27s_booby,అబోట్ యొక్క బూబీ (పాపాసుల అబోట్టి) సులిడ్ కుటు...,https://ebird.org/species/abbboo2,"Gangly, stretched-out-looking black-and-white ...","గ్యాంగ్లీ, సాగదీసిన నలుపు-తెలుపు సముద్రపు పక్ష...",https://eol.org/pages/45513856,Papasula (Abbott's Booby) is a genus of birds ...,పాపసుల (అబాట్స్ బూబీ) అనేది సులిడే కుటుంబానికి...,https://dibird.com/species/abbotts-booby/,https://www.iucnredlist.org/species/22696649/1...,https://www.wikidata.org/wiki/Q839854,Abbott's Booby.jpg,LC,T22696649A152726109
1,Abbott's Starling,అబాట్ యొక్క స్టార్లింగ్,Poeoptera femoralis,పూపెట్రా ఫెటోరిలిస్,అబోట్ యొక్క స్టార్లింగ్,"ఫొలియా ఫెమోరిలిస్, ఫోలిడాగ్స్ ఫెటోరిలిస్, సినీ...",Richmond,రిచ్ మండ్,1897.0,Animalia,జంతువులు,Chordata,చోర్డేటా,Aves,ఏవ్స్ (పక్షులు),Passeriformes,పాస్రిఫార్మ్స్,Sturnidae,స్టర్నిడైడ్,Poeoptera,పూపెట్రా,P. femoralis,పి. ఫెటోరిలిస్,అంతరించిపోతున్న జాతులు,"ఉష్ణమండల, భూమధ్యరేఖ",కెన్యా,టాంజానియా,NaN,తగ్గుతోంది,ఆఫ్రికా,కెన్యా మరియు టాంజానియా.,3.5 సంవత్సరాలు,"2,600 మీటర్లు",కార్నియా కళ్ళు,టిమ్పానిక్ మధ్య చెవి,5000 హెర్ట్జ్(Hz),10 డెసిబెల్స్ (dB),ద్విపాద కదలిక,అపటైట్,NaN,https://en.wikipedia.org/wiki/Abbott%27s_starling,అబోట్ యొక్క స్టార్లింగ్ (పోయోప్టెరా ఫెమోరాలిస్...,https://ebird.org/species/abbsta2,"A small, distinctive starling with a tiny rang...",కెన్యా మరియు టాంజానియాలో ఒక చిన్న శ్రేణితో ఒక ...,https://eol.org/pages/45515541,Poeoptera femoralis (Abbott's Starling) is a s...,పోయోప్టెరా ఫెమోరాలిస్ (అబాట్ యొక్క స్టార్లింగ్...,https://dibird.com/species/abbotts-starling/,https://www.iucnredlist.org/species/22710787/1...,https://www.wikidata.org/wiki/Q27075615,Abbott's Starling (adult).jpg,EN,T22710787A191046739
2,Abd al Kuri Sparrow,అబ్ద్ అల్ కురి స్పారో,Passer hemileucus,పాసర్ హెమిలేయుస్,"అబ్ద్ అల్ కురి స్పారో, అబ్ద్ అల్ కురి స్పారో",NaN,OgilvieGrant & Forbes,ఓగిల్వీగ్రాంట్ & ఫోర్బ్స్,1900.0,Animalia,జంతువులు,Chordata,చోర్డేటా,Aves,ఏవ్స్ (పక్షులు),Passeriformes,పాస్రిఫార్మ్స్,Passeridae,ప్యాసెరిడా,Passer,పాసర్,P. hemileucus,పి. హెమిలేయుస్,అంతరించిపోతున్న జాతులు,"స్క్రబ్లాండ్, రాకీ ప్రాంతాలు (ఉదా. లోతట్టు కొం...",సోమాలియా,NaN,NaN,స్థిరమైన,ఆఫ్రికా,"అబ్ద్ అల్-కురి I., సోకోట్రా యొక్క పడమర.",5.8 సంవత్సరాలు,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Abd_al_Kuri,అబ్ద్ అల్ కురి (అరబిక్: عبد الكوري) అనేది గార్...,NaN,NaN,NaN,NaN,NaN,NaN,https://dibird.com/species/abd-alkuri-sparrow/,https://www.iucnredlist.org/species/22735599/9...,https://www.wikidata.org/wiki/Q3056159,Passer hemileucus.jpg,EN,T22735599A95115321
3,Abdim's Stork,అబ్దిమ్ కొంగ,Ciconia abdimii,సికోనియా అబ్దిమిఐ,అ

In [34]:
for i in range(data.shape[0]):
    if not pd.isna(data.loc[i,'founded']):
        data.loc[i,'founded'] = str(int(data.loc[i,'founded']))

C:\Users\LEGION\AppData\Local\Temp\ipykernel_11396\98932758.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[i,'founded'] = str(int(data.loc[i,'founded']))


In [35]:
data.head(5)

,Bird Original Name,Bird Original Name Telugu,Scientific Name,Scientific Name Telugu,Synonyms Telugu,Old latin name for bird Telugu,Scientist,Scientist Telugu,founded,Kingdom,Kingdom Telugu,Phylum,Phylum Telugu,Class,Class Telugu,Order,Order Telugu,Family,Family Telugu,Genus,Genus Telugu,Species,Species Telugu,Conservation status Telugu,Habitat Telugu,Country Telugu,geographic distribution includes Telugu,biogeographic realm Telugu,Population Growth Telugu,Breeding region Telugu,Breeding subregion Telugu,Life Span Telugu,altitude Telugu,visual system Telugu,auditory system Telugu,hearing range Telugu,hearing threshold Telugu,locomotion Telugu,mineralized skeleton contains Telugu,body mass Telugu,Wiki_Link,Wiki_Description Telugu,Ebird_Link,Ebird_Description,Ebird_Description Telugu,Eol_Link,Eol_Description,Eol_Description Telugu,Dibird_Link,IUCN_link,Wikidata_link,image,Conservation status code,Page Id
0,Abbott's Booby,అబాట్ యొక్క బూబీ,Papasula abbotti,పాపసులా అబ్బోటి,అబోట్ యొక్క బూబీ,సుల అబ్బోటి,Ridgway,రిడ్వే,1893,Animalia,జంతువులు,Chordata,చోర్డేటా,Aves,ఏవ్స్ (పక్షులు),Suliformes,సులిఫార్మ్స్,Sulidae,సులిదా,"PapasulaOlson & Warheit, 1988",పాపసులా,P. abbotti,పి. అబ్బోటి,తక్కువ ఆందోళన జాతులు,"అటవీ, మెరైన్ నెరిటిక్, మెరైన్ ఓసియానిక్",క్రిస్మస్ ద్వీపం,NaN,NaN,స్థిరమైన,భారతీయ మహాసముద్రం,ఈశాన్య,"13,1 సంవత్సరాలు",260 మీటర్లు,కార్నియా కళ్ళు,టిమ్పానిక్ మధ్య చెవి,5000 హెర్ట్జ్(Hz),20 డెసిబెల్స్ (dB),ఎగరగల,అపటైట్,NaN,https://en.wikipedia.org/wiki/Abbott%27s_booby,అబోట్ యొక్క బూబీ (పాపాసుల అబోట్టి) సులిడ్ కుటు...,https://ebird.org/species/abbboo2,"Gangly, stretched-out-looking black-and-white ...","గ్యాంగ్లీ, సాగదీసిన నలుపు-తెలుపు సముద్రపు పక్ష...",https://eol.org/pages/45513856,Papasula (Abbott's Booby) is a genus of birds ...,పాపసుల (అబాట్స్ బూబీ) అనేది సులిడే కుటుంబానికి...,https://dibird.com/species/abbotts-booby/,https://www.iucnredlist.org/species/22696649/1...,https://www.wikidata.org/wiki/Q839854,Abbott's Booby.jpg,LC,T22696649A152726109
1,Abbott's Starling,అబాట్ యొక్క స్టార్లింగ్,Poeoptera femoralis,పూపెట్రా ఫెటోరిలిస్,అబోట్ యొక్క స్టార్లింగ్,"ఫొలియా ఫెమోరిలిస్, ఫోలిడాగ్స్ ఫెటోరిలిస్, సినీ...",Richmond,రిచ్ మండ్,1897,Animalia,జంతువులు,Chordata,చోర్డేటా,Aves,ఏవ్స్ (పక్షులు),Passeriformes,పాస్రిఫార్మ్స్,Sturnidae,స్టర్నిడైడ్,Poeoptera,పూపెట్రా,P. femoralis,పి. ఫెటోరిలిస్,అంతరించిపోతున్న జాతులు,"ఉష్ణమండల, భూమధ్యరేఖ",కెన్యా,టాంజానియా,NaN,తగ్గుతోంది,ఆఫ్రికా,కెన్యా మరియు టాంజానియా.,3.5 సంవత్సరాలు,"2,600 మీటర్లు",కార్నియా కళ్ళు,టిమ్పానిక్ మధ్య చెవి,5000 హెర్ట్జ్(Hz),10 డెసిబెల్స్ (dB),ద్విపాద కదలిక,అపటైట్,NaN,https://en.wikipedia.org/wiki/Abbott%27s_starling,అబోట్ యొక్క స్టార్లింగ్ (పోయోప్టెరా ఫెమోరాలిస్...,https://ebird.org/species/abbsta2,"A small, distinctive starling with a tiny rang...",కెన్యా మరియు టాంజానియాలో ఒక చిన్న శ్రేణితో ఒక ...,https://eol.org/pages/45515541,Poeoptera femoralis (Abbott's Starling) is a s...,పోయోప్టెరా ఫెమోరాలిస్ (అబాట్ యొక్క స్టార్లింగ్...,https://dibird.com/species/abbotts-starling/,https://www.iucnredlist.org/species/22710787/1...,https://www.wikidata.org/wiki/Q27075615,Abbott's Starling (adult).jpg,EN,T22710787A191046739
2,Abd al Kuri Sparrow,అబ్ద్ అల్ కురి స్పారో,Passer hemileucus,పాసర్ హెమిలేయుస్,"అబ్ద్ అల్ కురి స్పారో, అబ్ద్ అల్ కురి స్పారో",NaN,OgilvieGrant & Forbes,ఓగిల్వీగ్రాంట్ & ఫోర్బ్స్,1900,Animalia,జంతువులు,Chordata,చోర్డేటా,Aves,ఏవ్స్ (పక్షులు),Passeriformes,పాస్రిఫార్మ్స్,Passeridae,ప్యాసెరిడా,Passer,పాసర్,P. hemileucus,పి. హెమిలేయుస్,అంతరించిపోతున్న జాతులు,"స్క్రబ్లాండ్, రాకీ ప్రాంతాలు (ఉదా. లోతట్టు కొం...",సోమాలియా,NaN,NaN,స్థిరమైన,ఆఫ్రికా,"అబ్ద్ అల్-కురి I., సోకోట్రా యొక్క పడమర.",5.8 సంవత్సరాలు,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Abd_al_Kuri,అబ్ద్ అల్ కురి (అరబిక్: عبد الكوري) అనేది గార్...,NaN,NaN,NaN,NaN,NaN,NaN,https://dibird.com/species/abd-alkuri-sparrow/,https://www.iucnredlist.org/species/22735599/9...,https://www.wikidata.org/wiki/Q3056159,Passer hemileucus.jpg,EN,T22735599A95115321
3,Abdim's Stork,అబ్దిమ్ కొంగ,Ciconia abdimii,సికోనియా అబ్దిమిఐ,అబ్దిమ్

In [36]:
import sweetviz as sv

data.to_pickle('Dataset_final.pkl')
data.to_csv('Dataset_final.csv')
data.to_excel('Dataset_final.xlsx')
sv.analyze(data).show_html('report_final.html')

C:\Users\LEGION\anaconda3\envs\indicwiki\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Feature: Kingdom Telugu                      |██        | [ 20%]   00:03 -> (00:05 left)C:\Users\LEGION\anaconda3\envs\indicwiki\lib\site-packages\sweetviz\graph.py:33: UserWarning: Glyph 3100 (\N{TELUGU LETTER JA}) missing from current font.
  figure.savefig(as_raw_bytes, format='png', transparent=True)
C:\Users\LEGION\anaconda3\envs\indicwiki\lib\site-packages\sweetviz\graph.py:33: UserWarning: Matplotlib currently does not support Telugu natively.
  figure.savefig(as_raw_bytes, format='png', transparent=True)
C:\Users\LEGION\anaconda3\envs\indicwiki\lib\site-packages\sweetviz\graph.py:33: UserWarning: Glyph 3074 (\N{TELUGU SIGN ANUSVARA}) missing from current font.
  figure.savefig(as_raw_bytes, format='png', transparent=Tr

Report report_final.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [2]:
data = pd.read_excel('Dataset_final.xlsx')
data.head(5)

,Unnamed: 0,Bird Original Name,Bird Original Name Telugu,Scientific Name,Scientific Name Telugu,Synonyms Telugu,Old latin name for bird Telugu,Scientist,Scientist Telugu,founded,Kingdom,Kingdom Telugu,Phylum,Phylum Telugu,Class,Class Telugu,Order,Order Telugu,Family,Family Telugu,Genus,Genus Telugu,Species,Species Telugu,Conservation status Telugu,Habitat Telugu,Country Telugu,geographic distribution includes Telugu,biogeographic realm Telugu,Population Growth Telugu,Breeding region Telugu,Breeding subregion Telugu,Life Span Telugu,altitude Telugu,visual system Telugu,auditory system Telugu,hearing range Telugu,hearing threshold Telugu,locomotion Telugu,mineralized skeleton contains Telugu,body mass Telugu,Wiki_Link,Wiki_Description Telugu,Ebird_Link,Ebird_Description,Ebird_Description Telugu,Eol_Link,Eol_Description,Eol_Description Telugu,Dibird_Link,IUCN_link,Wikidata_link,image,Conservation status code,Page Id
0,0,Abbott's Booby,అబాట్ యొక్క బూబీ,Papasula abbotti,పాపసులా అబ్బోటి,అబోట్ యొక్క బూబీ,సుల అబ్బోటి,Ridgway,రిడ్వే,1893.0,Animalia,జంతువులు,Chordata,చోర్డేటా,Aves,ఏవ్స్ (పక్షులు),Suliformes,సులిఫార్మ్స్,Sulidae,సులిదా,"PapasulaOlson & Warheit, 1988",పాపసులా,P. abbotti,పి. అబ్బోటి,తక్కువ ఆందోళన జాతులు,"అటవీ, మెరైన్ నెరిటిక్, మెరైన్ ఓసియానిక్",క్రిస్మస్ ద్వీపం,NaN,NaN,స్థిరమైన,భారతీయ మహాసముద్రం,ఈశాన్య,"13,1 సంవత్సరాలు",260 మీటర్లు,కార్నియా కళ్ళు,టిమ్పానిక్ మధ్య చెవి,5000 హెర్ట్జ్(Hz),20 డెసిబెల్స్ (dB),ఎగరగల,అపటైట్,NaN,https://en.wikipedia.org/wiki/Abbott%27s_booby,అబోట్ యొక్క బూబీ (పాపాసుల అబోట్టి) సులిడ్ కుటు...,https://ebird.org/species/abbboo2,"Gangly, stretched-out-looking black-and-white ...","గ్యాంగ్లీ, సాగదీసిన నలుపు-తెలుపు సముద్రపు పక్ష...",https://eol.org/pages/45513856,Papasula (Abbott's Booby) is a genus of birds ...,పాపసుల (అబాట్స్ బూబీ) అనేది సులిడే కుటుంబానికి...,https://dibird.com/species/abbotts-booby/,https://www.iucnredlist.org/species/22696649/1...,https://www.wikidata.org/wiki/Q839854,Abbott's Booby.jpg,LC,T22696649A152726109
1,1,Abbott's Starling,అబాట్ యొక్క స్టార్లింగ్,Poeoptera femoralis,పూపెట్రా ఫెటోరిలిస్,అబోట్ యొక్క స్టార్లింగ్,"ఫొలియా ఫెమోరిలిస్, ఫోలిడాగ్స్ ఫెటోరిలిస్, సినీ...",Richmond,రిచ్ మండ్,1897.0,Animalia,జంతువులు,Chordata,చోర్డేటా,Aves,ఏవ్స్ (పక్షులు),Passeriformes,పాస్రిఫార్మ్స్,Sturnidae,స్టర్నిడైడ్,Poeoptera,పూపెట్రా,P. femoralis,పి. ఫెటోరిలిస్,అంతరించిపోతున్న జాతులు,"ఉష్ణమండల, భూమధ్యరేఖ",కెన్యా,టాంజానియా,NaN,తగ్గుతోంది,ఆఫ్రికా,కెన్యా మరియు టాంజానియా.,3.5 సంవత్సరాలు,"2,600 మీటర్లు",కార్నియా కళ్ళు,టిమ్పానిక్ మధ్య చెవి,5000 హెర్ట్జ్(Hz),10 డెసిబెల్స్ (dB),ద్విపాద కదలిక,అపటైట్,NaN,https://en.wikipedia.org/wiki/Abbott%27s_starling,అబోట్ యొక్క స్టార్లింగ్ (పోయోప్టెరా ఫెమోరాలిస్...,https://ebird.org/species/abbsta2,"A small, distinctive starling with a tiny rang...",కెన్యా మరియు టాంజానియాలో ఒక చిన్న శ్రేణితో ఒక ...,https://eol.org/pages/45515541,Poeoptera femoralis (Abbott's Starling) is a s...,పోయోప్టెరా ఫెమోరాలిస్ (అబాట్ యొక్క స్టార్లింగ్...,https://dibird.com/species/abbotts-starling/,https://www.iucnredlist.org/species/22710787/1...,https://www.wikidata.org/wiki/Q27075615,Abbott's Starling (adult).jpg,EN,T22710787A191046739
2,2,Abd al Kuri Sparrow,అబ్ద్ అల్ కురి స్పారో,Passer hemileucus,పాసర్ హెమిలేయుస్,"అబ్ద్ అల్ కురి స్పారో, అబ్ద్ అల్ కురి స్పారో",NaN,OgilvieGrant & Forbes,ఓగిల్వీగ్రాంట్ & ఫోర్బ్స్,1900.0,Animalia,జంతువులు,Chordata,చోర్డేటా,Aves,ఏవ్స్ (పక్షులు),Passeriformes,పాస్రిఫార్మ్స్,Passeridae,ప్యాసెరిడా,Passer,పాసర్,P. hemileucus,పి. హెమిలేయుస్,అంతరించిపోతున్న జాతులు,"స్క్రబ్లాండ్, రాకీ ప్రాంతాలు (ఉదా. లోతట్టు కొం...",సోమాలియా,NaN,NaN,స్థిరమైన,ఆఫ్రికా,"అబ్ద్ అల్-కురి I., సోకోట్రా యొక్క పడమర.",5.8 సంవత్సరాలు,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Abd_al_Kuri,అబ్ద్ అల్ కురి (అరబిక్: عبد الكوري) అనేది గార్...,NaN,NaN,NaN,NaN,NaN,NaN,https://dibird.com/species/abd-alkuri-sparrow/,https://www.iucnredlist.org/species/22735599/9...,https://www.wikidata.org/wiki/Q3056159,Passer hemileucus.jpg,EN,T22735599A95115321
3,3,Abdim's Stork,అబ్దిమ్ కొంగ,Ciconia abdimii,

In [ ]:
ఇవి ఎక్కువగా ఉపఉష్ణమండలాలలో నివసిస్తాయి. వీటి సంతానోత్పత్తి ప్రాంతం ఆస్ట్రేలియా. వీటి సంతానోత్పత్తి ఉపప్రాంతాలు సియా మరియు తహుళందాంగ్ (సులవేస్ కు ఈశాన్యంగా). సియావ్ పిట్టా జీవితకాలం సాధారణంగా 4.2 సంవత్సరాలుగా తెలుపబడినది.ఇవి భౌగోళికంగా 1,200 మీటర్లు ఎత్తు ప్రాంతాలవరకు జీవించగలవు.

In [86]:
data.set_index('Bird Original Name',inplace=True)


In [12]:
row_dict = {}
B = "Siau Pitta"
bird_dict = dict(data.loc[B])
for k,v in zip(bird_dict,bird_dict.values()):
    if not pd.isna(v):
        row_dict["_".join(k.split(' '))] = v
    else:
        row_dict["_".join(k.split(' '))] = 'NaN'
print(row_dict)

{'Unnamed:_0': 8280, 'Bird_Original_Name_Telugu': 'సియు పిట్టా', 'Scientific_Name': 'Erythropitta palliceps', 'Scientific_Name_Telugu': 'ఎరిట్రోపిట్ట పాలిసెప్స్', 'Synonyms_Telugu': 'సియావ్ పిట్టా', 'Old_latin_name_for_bird_Telugu': 'పిట్ట పాలిసెప్స్', 'Scientist': 'Brüggemann', 'Scientist_Telugu': 'బ్రూగ్గెమాన్', 'founded': 1876.0, 'Kingdom': 'Animalia', 'Kingdom_Telugu': 'జంతువులు', 'Phylum': 'Chordata', 'Phylum_Telugu': 'చోర్డేటా', 'Class': 'Aves', 'Class_Telugu': 'ఏవ్స్ (పక్షులు)', 'Order': 'Passeriformes', 'Order_Telugu': 'పాస్రిఫార్మ్స్', 'Family': 'Pittidae', 'Family_Telugu': 'పిట్టిడా', 'Genus': 'Erythropitta', 'Genus_Telugu': 'ఎరిట్రోపిట్ట', 'Species': 'E.\xa0palliceps', 'Species_Telugu': 'ఇ. పాలిసెప్స్', 'Conservation_status_Telugu': 'NaN', 'Habitat_Telugu': 'ఉపఉష్ణమండల', 'Country_Telugu': 'ఇండోనేషియా', 'geographic_distribution_includes_Telugu': 'NaN', 'biogeographic_realm_Telugu': 'NaN', 'Population_Growth_Telugu': 'తగ్గుతోంది', 'Breeding_region_Telugu': 'ఆస్ట్రేలియా', 'Bree

In [1]:
file_loader = FileSystemLoader('./Template/')
env = Environment(loader=file_loader)
template = env.get_template('birds_jeevanavidanam.j2')

NameError: name 'FileSystemLoader' is not defined

In [88]:
print(template.render(row_dict))

వీటి నివాస స్థలం గ్రాస్విట్ భూమి, భూగోళం. వీటి సంతానోత్పత్తి ప్రాంతం దక్షిణ అమెరికా. వీటి సంతానోత్పత్తి ఉపప్రాంతాలు పడమర. పర్వత చిలుక జీవితకాలం సాధారణంగా 6 సంవత్సరాలుగా తెలుపబడినది.వీటికి తగిన ఆక్సిజన్ 4,500 మీటర్లుఎత్తు వరకు లభిస్తుంది. 


In [21]:
def rd(B):
    row_dict = {}
    bird_dict = dict(data.loc[B])
    for k,v in zip(bird_dict,bird_dict.values()):
        if not pd.isna(v):
            row_dict["_".join(k.split(' '))] = v
        else:
            row_dict["_".join(k.split(' '))] = 'NaN'
    if row_dict["founded"] != 'NaN':
        row_dict["founded"] = str(int(row_dict["founded"]))
    row_dict['Bird_Original_Name'] = B
    return row_dict


In [124]:
row_dict = rd("Bay Wren")

In [109]:
print(row_dict)

{'Unnamed:_0': 6744.0, 'Bird_Original_Name_Telugu': 'పర్వత చిలుక', 'Scientific_Name': 'Psilopsiagon aurifrons', 'Scientific_Name_Telugu': 'సైలోప్సియాగాన్ ఆరిఫ్రాన్స్', 'Synonymous_Name': 'Mountain Parakeet', 'Synonymous_Name_Telugu': 'మౌంటైన్ పారాకీట్', 'Unnamed:_7': 'NaN', 'Old_latin_name_for_bird_Telugu': 'బోల్బోరిన్చుస్ ఆరిఫ్రాన్స్, బోలోబోర్ ఆరిఫ్రాన్స్, సిట్టకస్ (లతామస్) ఆరిఫ్రాన్లు', 'Scientist': 'Lesson', 'Scientist_Telugu': 'పాఠం', 'founded': '1830', 'Kingdom': 'Animalia', 'Kingdom_Telugu': 'జంతువులు', 'Phylum': 'Chordata', 'Phylum_Telugu': 'చోర్డేటా', 'Class': 'Aves', 'Class_Telugu': 'ఏవ్స్ (పక్షులు)', 'Order': 'Psittaciformes', 'Order_Telugu': 'పిట్టసిఫార్మ్స్', 'Family': 'Psittacidae', 'Family_Telugu': 'సిట్టాసిడే', 'Genus': 'Psilopsiagon', 'Genus_Telugu': 'సైలోప్సియాగాన్', 'Species': 'P.\xa0aurifrons', 'Species_Telugu': 'పి. ఆరిఫ్రాన్స్', 'Conservation_status_Telugu': 'తక్కువ ఆందోళన జాతులు', 'Habitat_Telugu': 'గ్రాస్విట్ భూమి, భూగోళం', 'Country_Telugu': 'NaN', 'geographic_di

In [102]:
template = env.get_template('birds_gurthimpu.j2')

In [103]:
print(template.render(row_dict))

ఈ పక్షిని ప్రకృతి శాస్త్రవేత్త పాఠం గారు 1830 లో కనుగొన్నారు.


In [110]:
template = env.get_template('birds_intro.j2')

In [111]:
print(template.render(row_dict))

పర్వత చిలుక(Mountain Parakeet)సైలోప్సియాగాన్ ఆరిఫ్రాన్స్(Psilopsiagon aurifrons) దీని శాస్త్రీయ నామం ,సిట్టాసిడే కుటుంబంలోని పక్షి.  పక్షికి లాటిన్ పేరు(పాత) బోల్బోరిన్చుస్ ఆరిఫ్రాన్స్, బోలోబోర్ ఆరిఫ్రాన్స్, సిట్టకస్ (లతామస్) ఆరిఫ్రాన్లు.
	దీనిని మౌంటైన్ పారాకీట్ ( Mountain Parakeet ) అని కూడా అంటారు .ఇది జంతువులు జాతికి చెందినది .పర్వత చిలుక యొక్క  ఉపరాజ్యం బిలేటేరియా మరియు డ్యూటెరోస్టోమియా దీని ఇన్ఫ్రా కింగ్డమ్  
	ఈ పక్షి యొక్క ఫైలం చోర్డేటా, సబ్ ఫైలం వెన్నుపూస మరియు ఇన్ఫ్రా ఫైలం గ్నాథోస్టోమాటా. ఇవి పక్షులు తరగతి కి చెందినవి 
	ఈ పక్షి యొక్క ఆర్డర్ పిట్టసిఫార్మ్స్. ఐ.యు.సి.ఎన్. ఈ జాతులను తక్కువ ఆందోళన జాతులు పక్షిగా వర్గీకరించింది.


In [22]:
data = pd.read_excel('data/Dataset_final.xlsx')
data.set_index('Bird Original Name',inplace=True)


In [150]:
# template = env.get_template('birds_lakshanalu.j2')
# print(template.render(row_dict))

In [4]:
data.head(4)

,Unnamed: 0,Bird Original Name Telugu,Scientific Name,Scientific Name Telugu,Synonymous Name,Synonymous Name Telugu,Unnamed: 7,Old latin name for bird Telugu,Scientist,Scientist Telugu,founded,Kingdom,Kingdom Telugu,Phylum,Phylum Telugu,Class,Class Telugu,Order,Order Telugu,Family,Family Telugu,Genus,Genus Telugu,Species,Species Telugu,Conservation status Telugu,Habitat Telugu,Country Telugu,geographic distribution includes Telugu,biogeographic realm Telugu,Population Growth Telugu,Breeding region Telugu,Breeding subregion Telugu,Life Span Telugu,altitude Telugu,visual system Telugu,auditory system Telugu,hearing range Telugu,hearing threshold Telugu,locomotion Telugu,mineralized skeleton contains Telugu,body mass Telugu,Wiki_Link,Wiki_Description Telugu,Ebird_Link,Ebird_Description,Ebird_Description Telugu,Eol_Link,Eol_Description,Eol_Description Telugu,Dibird_Link,IUCN_link,Wikidata_link,image,Conservation status code,Page Id,Wikidata Id
Bird Original Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abbott's Booby,1.0,అబాట్ బూబీ,Papasula abbotti,పాపసులా అబ్బోటి,Abbott's Booby,అబాట్ బూబీ,NaN,సుల అబ్బోటి,Ridgway,రిడ్వే,1893.0,Animalia,జంతువులు,Chordata,చోర్డేటా,Aves,ఏవ్స్ (పక్షులు),Suliformes,సులిఫార్మ్స్,Sulidae,సులిదా,"PapasulaOlson & Warheit, 1988",పాపసులా,P. abbotti,పి. అబ్బోటి,తక్కువ ఆందోళన జాతులు,"అటవీ, మెరైన్ నెరిటిక్, మెరైన్ ఓసియానిక్",క్రిస్మస్ ద్వీపం,NaN,NaN,స్థిరమైన,ఇండియన్ మహాసముద్రం,ఈశాన్య,13.1 సంవత్సరాలు,260 మీటర్లు,కార్నియా కళ్ళు,టిమ్పానిక్ మధ్య చెవి,5000 హెర్ట్జ్(Hz),20 డెసిబెల్స్ (dB),ఎగరగల,అపటైట్,NaN,https://en.wikipedia.org/wiki/Abbott%27s_booby,అబోట్ బూబీ (పాపాసుల అబోట్టి) సులిడ్ కుటుంబానిక...,https://ebird.org/species/abbboo2,"Gangly, stretched-out-looking black-and-white ...","గ్యాంగ్లీ, సాగదీసిన నలుపు-మరియు-తెలుపు సీపక్షి...",https://eol.org/pages/45513856,Papasula (Abbott's Booby) is a genus of birds ...,పాపసుల (అబాట్స్ బూబీ) అనేది సులిడే కుటుంబానికి...,https://dibird.com/species/abbotts-booby/,https://www.iucnredlist.org/species/22696649/1...,https://www.wikidata.org/wiki/Q839854,Abbott's Booby.jpg,LC,T22696649A152726109,Q839854
Abbott's Starling,2.0,అబాట్ స్టార్లింగ్,Poeoptera femoralis,పూపెట్రా ఫెటోరిలిస్,Abbott's Starling,అబాట్ స్టార్లింగ్,NaN,"ఫొలియా ఫెమోరిలిస్, ఫోలిడాగ్స్ ఫెటోరిలిస్, సినీ...",Richmond,రిచ్ మండ్,1897.0,Animalia,జంతువులు,Chordata,చోర్డేటా,Aves,ఏవ్స్ (పక్షులు),Passeriformes,పాస్రిఫార్మ్స్,Sturnidae,స్టర్నిడైడ్,Poeoptera,పూపెట్రా,P. femoralis,పి. ఫెటోరిలిస్,అంతరించిపోతున్న జాతులు,"ఉష్ణమండల, భూమధ్యరేఖ",కెన్యా,టాంజానియా,NaN,తగ్గుతోంది,ఆఫ్రికా,కెన్యా మరియు టాంజానియా,3.5 సంవత్సరాలు,"2,600 మీటర్లు",కార్నియా కళ్ళు,టిమ్పానిక్ మధ్య చెవి,5000 హెర్ట్జ్(Hz),10 డెసిబెల్స్ (dB),ద్విపాద కదలిక,అపటైట్,NaN,https://en.wikipedia.org/wiki/Abbott%27s_starling,అబోట్ స్టార్లింగ్ (పోయోప్టెరా ఫెమోరాలిస్) అనేద...,https://ebird.org/species/abbsta2,కెన్యా మరియు టాంజానియాలో ఒక చిన్న శ్రేణితో ఒక ...,కెన్యా మరియు టాంజానియాలో ఒక చిన్న శ్రేణితో ఒక ...,https://eol.org/pages/45515541,Poeoptera femoralis (Abbott's Starling) is a s...,పోయోప్టెరా ఫెమోరాలిస్ (అబాట్ యొక్క స్టార్లింగ్...,https://dibird.com/species/abbotts-starling/,https://www.iucnredlist.org/species/22710787/1...,https://www.wikidata.org/wiki/Q27075615,Abbott's Starling (adult).jpg,EN,T22710787A191046739,Q27075615
Abd al Kuri Sparrow,3.0,అబ్ద్ అల్ కురి స్పారో,Passer hemileucus,పాసర్ హెమిలేయుస్,"Abd al Kuri Sparrow, Abd Al Kuri Sparrow","అబ్ద్ అల్ కురి స్పారో, అబ్ద్ అల్ కురి స్పారో",NaN,NaN,OgilvieGrant & Forbes,ఓగిల్వీగ్రాంట్ & ఫోర్బ్స్,1900.0,Animalia,జంతువులు,Chordata,చోర్డేటా,Aves,ఏవ్స్ (పక్షులు),Passeriformes,పాస్రిఫార్మ్స్,Passeridae,ప్యాసెరిడా,Passer,పాసర్,P. hemileucus,పి. హెమిలేయుస్,అంతరించిపోతున్న జాతులు,"స్క్రబ్లాండ్, రాకీ ప్రాంతాలు (ఉదా. లోతట్టు కొం...",సోమాలియా,NaN,NaN,స్థిరమైన,ఆఫ్రికా,"అబ్ద్ అల్-కురి I, సోకోట్రా యొక్క పడమర",5.8 సంవత్సరాలు,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Abd_al_Kuri,అబ్ద్ అల్ కురి (అరబిక్: عبد الكوري) అనేది గార్...,NaN,NaN,NaN,NaN,NaN,NaN,https://dibird.com/species/abd-alkuri-sparrow/,h

In [151]:
print(row_dict)

{'Unnamed:_0': 7770.0, 'Bird_Original_Name_Telugu': 'ఫిల్బీస్ పార్ట్రిడ్జ్', 'Scientific_Name': 'Alectoris philbyi', 'Scientific_Name_Telugu': 'అలెక్టర్లు ఫిల్బై', 'Synonymous_Name': "Philby's Partridge, Philby's Rock Partridge", 'Synonymous_Name_Telugu': 'ఫిల్బీ   పార్ట్రిడ్జ్, ఫిల్బీ   రాక్ పార్ట్రిడ్జ్', 'Unnamed:_7': 'NaN', 'Old_latin_name_for_bird_Telugu': 'అలెక్టర్లు ఫిల్బీ', 'Scientist': 'Lowe', 'Scientist_Telugu': 'లొవె ', 'founded': '1934', 'Kingdom': 'Animalia', 'Kingdom_Telugu': 'జంతువులు', 'Phylum': 'Chordata', 'Phylum_Telugu': 'చోర్డేటా', 'Class': 'Aves', 'Class_Telugu': 'ఏవ్స్ (పక్షులు)', 'Order': 'Galliformes', 'Order_Telugu': 'గల్లిఫార్మ్స్', 'Family': 'Phasianidae', 'Family_Telugu': 'ఫిసియానిడే', 'Genus': 'Alectoris', 'Genus_Telugu': 'అలెక్ట్రిస్', 'Species': 'A.\xa0philbyi', 'Species_Telugu': 'ఎ. ఫిల్బై', 'Conservation_status_Telugu': 'తక్కువ ఆందోళన జాతులు', 'Habitat_Telugu': 'భూవాసి (నెలకు చెందినది)', 'Country_Telugu': 'సౌదీ అరేబియా', 'geographic_distribution_include

In [152]:
data.head(5)

,Unnamed: 0,Bird Original Name Telugu,Scientific Name,Scientific Name Telugu,Synonymous Name,Synonymous Name Telugu,Unnamed: 7,Old latin name for bird Telugu,Scientist,Scientist Telugu,founded,Kingdom,Kingdom Telugu,Phylum,Phylum Telugu,Class,Class Telugu,Order,Order Telugu,Family,Family Telugu,Genus,Genus Telugu,Species,Species Telugu,Conservation status Telugu,Habitat Telugu,Country Telugu,geographic distribution includes Telugu,biogeographic realm Telugu,Population Growth Telugu,Breeding region Telugu,Breeding subregion Telugu,Life Span Telugu,altitude Telugu,visual system Telugu,auditory system Telugu,hearing range Telugu,hearing threshold Telugu,locomotion Telugu,mineralized skeleton contains Telugu,body mass Telugu,Wiki_Link,Wiki_Description Telugu,Ebird_Link,Ebird_Description,Ebird_Description Telugu,Eol_Link,Eol_Description,Eol_Description Telugu,Dibird_Link,IUCN_link,Wikidata_link,image,Conservation status code,Page Id,Wikidata Id
Bird Original Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abbott's Booby,1.0,అబాట్ బూబీ,Papasula abbotti,పాపసులా అబ్బోటి,Abbott's Booby,అబాట్ బూబీ,NaN,సుల అబ్బోటి,Ridgway,రిడ్వే,1893.0,Animalia,జంతువులు,Chordata,చోర్డేటా,Aves,ఏవ్స్ (పక్షులు),Suliformes,సులిఫార్మ్స్,Sulidae,సులిదా,"PapasulaOlson & Warheit, 1988",పాపసులా,P. abbotti,పి. అబ్బోటి,తక్కువ ఆందోళన జాతులు,"అటవీ, మెరైన్ నెరిటిక్, మెరైన్ ఓసియానిక్",క్రిస్మస్ ద్వీపం,NaN,NaN,స్థిరమైన,ఇండియన్ మహాసముద్రం,ఈశాన్య,13.1 సంవత్సరాలు,260 మీటర్లు,కార్నియా కళ్ళు,టిమ్పానిక్ మధ్య చెవి,5000 హెర్ట్జ్(Hz),20 డెసిబెల్స్ (dB),ఎగరగల,అపటైట్,NaN,https://en.wikipedia.org/wiki/Abbott%27s_booby,అబోట్ బూబీ (పాపాసుల అబోట్టి) సులిడ్ కుటుంబానిక...,https://ebird.org/species/abbboo2,"Gangly, stretched-out-looking black-and-white ...","గ్యాంగ్లీ, సాగదీసిన నలుపు-మరియు-తెలుపు సీపక్షి...",https://eol.org/pages/45513856,Papasula (Abbott's Booby) is a genus of birds ...,పాపసుల (అబాట్స్ బూబీ) అనేది సులిడే కుటుంబానికి...,https://dibird.com/species/abbotts-booby/,https://www.iucnredlist.org/species/22696649/1...,https://www.wikidata.org/wiki/Q839854,Abbott's Booby.jpg,LC,T22696649A152726109,Q839854
Abbott's Starling,2.0,అబాట్ స్టార్లింగ్,Poeoptera femoralis,పూపెట్రా ఫెటోరిలిస్,Abbott's Starling,అబాట్ స్టార్లింగ్,NaN,"ఫొలియా ఫెమోరిలిస్, ఫోలిడాగ్స్ ఫెటోరిలిస్, సినీ...",Richmond,రిచ్ మండ్,1897.0,Animalia,జంతువులు,Chordata,చోర్డేటా,Aves,ఏవ్స్ (పక్షులు),Passeriformes,పాస్రిఫార్మ్స్,Sturnidae,స్టర్నిడైడ్,Poeoptera,పూపెట్రా,P. femoralis,పి. ఫెటోరిలిస్,అంతరించిపోతున్న జాతులు,"ఉష్ణమండల, భూమధ్యరేఖ",కెన్యా,టాంజానియా,NaN,తగ్గుతోంది,ఆఫ్రికా,కెన్యా మరియు టాంజానియా,3.5 సంవత్సరాలు,"2,600 మీటర్లు",కార్నియా కళ్ళు,టిమ్పానిక్ మధ్య చెవి,5000 హెర్ట్జ్(Hz),10 డెసిబెల్స్ (dB),ద్విపాద కదలిక,అపటైట్,NaN,https://en.wikipedia.org/wiki/Abbott%27s_starling,అబోట్ స్టార్లింగ్ (పోయోప్టెరా ఫెమోరాలిస్) అనేద...,https://ebird.org/species/abbsta2,కెన్యా మరియు టాంజానియాలో ఒక చిన్న శ్రేణితో ఒక ...,కెన్యా మరియు టాంజానియాలో ఒక చిన్న శ్రేణితో ఒక ...,https://eol.org/pages/45515541,Poeoptera femoralis (Abbott's Starling) is a s...,పోయోప్టెరా ఫెమోరాలిస్ (అబాట్ యొక్క స్టార్లింగ్...,https://dibird.com/species/abbotts-starling/,https://www.iucnredlist.org/species/22710787/1...,https://www.wikidata.org/wiki/Q27075615,Abbott's Starling (adult).jpg,EN,T22710787A191046739,Q27075615
Abd al Kuri Sparrow,3.0,అబ్ద్ అల్ కురి స్పారో,Passer hemileucus,పాసర్ హెమిలేయుస్,"Abd al Kuri Sparrow, Abd Al Kuri Sparrow","అబ్ద్ అల్ కురి స్పారో, అబ్ద్ అల్ కురి స్పారో",NaN,NaN,OgilvieGrant & Forbes,ఓగిల్వీగ్రాంట్ & ఫోర్బ్స్,1900.0,Animalia,జంతువులు,Chordata,చోర్డేటా,Aves,ఏవ్స్ (పక్షులు),Passeriformes,పాస్రిఫార్మ్స్,Passeridae,ప్యాసెరిడా,Passer,పాసర్,P. hemileucus,పి. హెమిలేయుస్,అంతరించిపోతున్న జాతులు,"స్క్రబ్లాండ్, రాకీ ప్రాంతాలు (ఉదా. లోతట్టు కొం...",సోమాలియా,NaN,NaN,స్థిరమైన,ఆఫ్రికా,"అబ్ద్ అల్-కురి I, సోకోట్రా యొక్క పడమర",5.8 సంవత్సరాలు,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Abd_al_Kuri,అబ్ద్ అల్ కురి (అరబిక్: عبد الكوري) అనేది గార్...,NaN,NaN,NaN,NaN,NaN,NaN,https://dibird.com/species/abd-alkuri-sparrow/,h

In [35]:
file_loader = FileSystemLoader('./Template/')
env = Environment(loader=file_loader)

In [13]:
template = env.get_template('birds.j2')
Bi = "Philby's Partridge"
row_dict = rd(Bi)
print(template.render(row_dict))

{{taxobox
| name= ఫిల్బీస్ పార్ట్రిడ్జ్
| status = LC
| status_system = IUCN3.1
| status_ref=<ref name="iucn status 18 November 2021">{{cite iucn |author=BirdLife International |date=2016 |title=''Philby's Partridge'' |volume=2016 |page=e.T22678698A92784515 |doi=10.2305/IUCN.UK.2016-3.RLTS.T22678698A92784515.en |access-date=18 November 2021}}</ref>
| image = Philby-Steinhuhn.jpg
| regnum= జంతువులు
| subregnum = బిలేటేరియా
| infraregnum = డ్యూటెరోస్టోమియా
| phylum= కార్డేటా
| subphylum  = సకశేరుకాలు
| infraphylum = గ్నాథోస్టోమాటా
| superclassis = టెట్రాపోడా
| classis = పక్షులు
| ordo=గల్లిఫార్మ్స్
| familia=ఫిసియానిడే
| genus =''అలెక్ట్రిస్''
| species = '''''ఎ. ఫిల్బై'''''
| binomial = అలెక్టర్లు ఫిల్బై
| authority = ([[లొవె ]], 1934)
| synonyms =  
* ''ఫిల్బీస్ పార్ట్రిడ్జ్''
}}


'''ఫిల్బీస్ పార్ట్రిడ్జ్''' (Philby's Partridge)  '''అలెక్టర్లు ఫిల్బై''' ( ''Alectoris philbyi'' ) దీని శాస్త్రీయ నామం.  ఇది ఫిసియానిడే కుటుంబానికి చెందిన పక్షి. పక్షికి లాటిన్ పేరు(పాత) అలెక్టర్లు ఫిల్బీ. 

In [24]:
import ast
from jinja2 import Environment, FileSystemLoader,Template
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

data = pd.read_excel('data/Dataset_final.xlsx')
birds = data.copy()
birds.drop(columns=['Unnamed: 0'],inplace=True)
data.set_index('Bird Original Name',inplace=True)
birds.reset_index(inplace=True)

def rd(B):
    row_dict = {}
    bird_dict = dict(data.loc[B])
    for k,v in zip(bird_dict,bird_dict.values()):
        if not pd.isna(v) :
            row_dict["_".join(k.split(' '))] = str(v).strip()
        else:
            row_dict["_".join(k.split(' '))] = 'NaN'
    if row_dict["founded"] != 'NaN':
        row_dict["founded"] = str(int(float(row_dict["founded"])))
    row_dict['Bird_Original_Name'] = B
    return row_dict

file_loader = FileSystemLoader('./Template/')
env = Environment(loader=file_loader)

In [31]:
env.globals.update(zip=zip)
template = env.get_template('birds.j2')


In [26]:
birds.head(5)

,index,Bird Original Name,Bird Original Name Telugu,Scientific Name,Scientific Name Telugu,Synonymous Name,Synonymous Name Telugu,Old latin name for bird,Old latin name for bird Telugu,Scientist,Scientist Telugu,founded,Kingdom,Kingdom Telugu,Phylum,Phylum Telugu,Class,Class Telugu,Order,Order Telugu,Family,Family Telugu,Genus,Genus Telugu,Species,Species Telugu,Conservation status Telugu,Habitat Telugu,Country Telugu,geographic distribution includes Telugu,biogeographic realm Telugu,Population Growth Telugu,Breeding region Telugu,Breeding subregion Telugu,Life Span Telugu,altitude Telugu,visual system Telugu,auditory system Telugu,hearing range Telugu,hearing threshold Telugu,locomotion Telugu,mineralized skeleton contains Telugu,body mass Telugu,Wiki_Link,Wiki_Description Telugu,Ebird_Link,Ebird_Description,Ebird_Description Telugu,Eol_Link,Eol_Description,Eol_Description Telugu,Dibird_Link,IUCN_link,Wikidata_link,image,Conservation status code,Page Id,Wikidata Id
0,0,Abbott's Booby,అబాట్ బూబీ,Papasula abbotti,పాపసులా అబ్బోటి,NaN,NaN,Sula abbotti,సుల అబ్బోటి,Ridgway,రిడ్వే,1893.0,Animalia,జంతువులు,Chordata,కార్డేటా,Aves,ఏవ్స్ (పక్షులు),Suliformes,సులిఫార్మ్స్,Sulidae,సులిదా,"PapasulaOlson & Warheit, 1988",పాపసులా,P. abbotti,పి. అబ్బోటి,అంతరించిపోతున్న జాతులు,"అటవీ, మెరైన్ నెరిటిక్, మెరైన్ ఓసియానిక్",క్రిస్మస్ ద్వీపం,NaN,NaN,స్థిరమైన,ఇండియన్ మహాసముద్రం,ఈశాన్య ప్రాంతాలు,14.1 సంవత్సరాలు,260 మీటర్లు,కార్నియా కళ్ళు,టిమ్పానిక్ మధ్య చెవి,5000 హెర్ట్జ్(Hz),20 డెసిబెల్స్ (dB),ఎగరడం,అపటైట్,NaN,https://en.wikipedia.org/wiki/Abbott%27s_booby,అబోట్ బూబీ (పాపాసుల అబోట్టి) సులిడ్ కుటుంబానిక...,https://ebird.org/species/abbboo2,"Gangly, stretched-out-looking black-and-white ...",నలుపు ఇంకా తెలుపు రంగుల సముద్రపు పక్షి. క్రిస్...,https://eol.org/pages/45513856,Papasula (Abbott's Booby) is a genus of birds ...,పాపసుల (అబాట్స్ బూబీ) అనేది సులిడే కుటుంబానికి...,https://dibird.com/species/abbotts-booby/,https://www.iucnredlist.org/species/22696649/1...,https://www.wikidata.org/wiki/Q839854,Abbott's Booby.jpg,EN,T22696649A152726109,Q839854
1,1,Abbott's Starling,అబాట్ స్టార్లింగ్,Poeoptera femoralis,పూపెట్రా ఫెటోరిలిస్,NaN,NaN,"Pholia femoralis, Pholidauges femoralis, Cinny...","ఫొలియా ఫెమోరిలిస్, ఫోలిడాగ్స్ ఫెటోరిలిస్, సినీ...",Richmond,రిచ్ మండ్,1897.0,Animalia,జంతువులు,Chordata,కార్డేటా,Aves,ఏవ్స్ (పక్షులు),Passeriformes,పాస్రిఫార్మ్స్,Sturnidae,స్టర్నిడైడ్,Poeoptera,పూపెట్రా,P. femoralis,పి. ఫెటోరిలిస్,అంతరించిపోతున్న జాతులు,"ఉష్ణమండల, భూమధ్యరేఖ",కెన్యా,టాంజానియా,NaN,తగ్గుతోంది,ఆఫ్రికా ఖండం,"కెన్యా ,టాంజానియా",3.5 సంవత్సరాలు,"2,600 మీటర్లు",కార్నియా కళ్ళు,టిమ్పానిక్ మధ్య చెవి,5000 హెర్ట్జ్(Hz),10 డెసిబెల్స్ (dB),ద్విపాద కదలిక,అపటైట్,NaN,https://en.wikipedia.org/wiki/Abbott%27s_starling,అబోట్ స్టార్లింగ్ (పోయోప్టెరా ఫెమోరాలిస్) అనేద...,https://ebird.org/species/abbsta2,NaN,"కెన్యా, టాంజానియాలో ఒక చిన్న శ్రేణితో ఒక చిన్న...",https://eol.org/pages/45515541,Poeoptera femoralis (Abbott's Starling) is a s...,పోయోప్టెరా ఫెమోరాలిస్ (అబాట్ స్టార్లింగ్) అనే...,https://dibird.com/species/abbotts-starling/,https://www.iucnredlist.org/species/22710787/1...,https://www.wikidata.org/wiki/Q27075615,Abbott's Starling (adult).jpg,EN,T22710787A191046739,Q27075615
2,2,Abd al Kuri Sparrow,అబ్ద్ అల్ కురి స్పారో,Passer hemileucus,పాసర్ హెమిలేయుస్,Abd al Kuri Sparrow,అబ్ద్ అల్ కురి స్పారో,NaN,NaN,OgilvieGrant & Forbes,ఓగిల్వీగ్రాంట్ & ఫోర్బ్స్,1900.0,Animalia,జంతువులు,Chordata,కార్డేటా,Aves,ఏవ్స్ (పక్షులు),Passeriformes,పాస్రిఫార్మ్స్,Passeridae,ప్యాసెరిడా,Passer,పాసర్,P. hemileucus,పి. హెమిలేయుస్,భవిషత్తులో అంతరించిపోయే అవకాశం వున్నా జాతులు,"స్క్రబ్లాండ్, రాకీ ప్రాంతాలు (ఉదా. లోతట్టు కొం...",సోమాలియా,NaN,NaN,స్థిరమైన,ఆఫ్రికా ఖండం,"అబ్ద్ అల్-కురి I, సోకోట్రా పడమర",5.8 సంవత్సరాలు,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Abd_al_Kuri,అబ్ద్ అల్ కురి (అరబిక్: عبد الكوري) అనేది గార్...,NaN,NaN,NaN,NaN,NaN,NaN,https://dibird.com/species/abd-alkuri-sparrow/,https://www.iucnredlist.org/species/22735599/9...,https://www.wikidata.org/wiki/Q3056159,Passer hem

In [27]:
# birds.drop(columns=['Unnamed: 0'],inplace=True)
# data.head(4)

In [28]:
birds.reset_index(inplace=True)
birds['Bird Original Name'][0]

"Abbott's Booby"

In [29]:
def clean_format(text):
    text = text.replace('..',".")
    text = text.replace(',.',".")
    text = text.replace('.,',".")
    text = text.replace(' . ',". ")
    text = text.replace(' , ',", ")
    text = text.replace(' . ',". ")
    text = text.replace(' , ',", ")
    return text


In [33]:
template = env.get_template('birds.j2')
Bi = "Rufous Coucal"
row_dict = rd(Bi)
print(clean_format(template.render(row_dict)))

{{taxobox
| name= రూఫస్ కౌకల్
| status = NT
| status_system = IUCN3.1
| status_ref=<ref name="iucn status 18 November 2021">{{cite iucn |author=BirdLife International |date=2016 |title=''Rufous Coucal'' |volume=2016 |page=e.T22684306A111308785 |doi=10.2305/IUCN.UK.2016-3.RLTS.T22684306A111308785.en |access-date=18 November 2021}}</ref>
| image = Rufous-coucal.jpg
| regnum= జంతువులు
| subregnum = బిలేటేరియా
| infraregnum = డ్యూటెరోస్టోమియా
| phylum= కార్డేటా
| subphylum  = సకశేరుకాలు
| infraphylum = గ్నాథోస్టోమాటా
| superclassis = టెట్రాపోడా
| classis = పక్షులు
| ordo=కుకులిఫార్మ్స్
| familia=కుకులిడే
| genus =''సెంటర్పస్''
| species = '''''సి. ఉనిరుబుస్'''''
| binomial = సెంటర్పస్ ఉనిరుఫస్
| authority = ([[క్యాబనిస్ & హీన్]], 1863)
}}


'''రూఫస్ కౌకల్''' (Rufous Coucal)  దీని శాస్త్రీయ నామం '''''సెంటర్పస్ ఉనిరుఫస్''''' (''Centropus unirufus'').  ఇది కుకులిడే కుటుంబానికి చెందిన పక్షి. పూర్వం లాటిన్ భాషలో పిరోసెంటోర్ ఉనిరుబుస్ అని పిలవబడేది. ఈ పక్షి వర్గం కార్డేటా, ఉపవర్గం సకశేరుకాలు, ని